In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Normalize Job Titles") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Đọc file CSV với tham số xử lý lỗi
job_data = spark.read.option("header", "true") \
    .option("mode", "PERMISSIVE") \
    .option("multiLine", "true") \
    .option("escape", "\\") \
    .option("quote", "\"") \
    .csv('data/jobs_data_careerviet2_final.csv')

# Đọc file danh sách các tiêu đề công việc chuẩn bằng tiếng Anh
# standard_titles_df = spark.read.option("header", "true").csv('analysis/job_titles_output.csv')
# standard_titles = [row["Job Title"] for row in standard_titles_df.collect()]

# Ánh xạ tiêu đề tiếng Việt sang tiếng Anh
# Chỉ sử dụng các tên trong danh sách cung cấp
professions = [
  { "name": "University Teacher", "keywords": ["University", "Academic", "Education", "Professor", "Teaching"] },
  { "name": "College Teacher", "keywords": ["College", "Academic", "Education", "Instructor", "Teaching"] },
  { "name": "Secondary Vocational and Professional School Teacher", "keywords": ["Vocational", "Professional", "Education", "Teaching", "Skills"] },
  { "name": "Upper Secondary Education Teacher", "keywords": ["Secondary", "Education", "Teaching", "High School", "Curriculum"] },
  { "name": "Lower Secondary Education Teacher", "keywords": ["Secondary", "Education", "Teaching", "Middle School", "Youth"] },
  { "name": "Foreign Language Teacher", "keywords": ["Language", "Teaching", "Linguistic", "International", "Communication"] },
  { "name": "Public Relations Officer", "keywords": ["PR", "Communication", "Media", "Corporate", "Reputation"] },
  { "name": "Lawyer", "keywords": ["Legal", "Law", "Attorney", "Advocate", "Justice"] },
  { "name": "Archivist and Curator", "keywords": ["Archive", "Preservation", "Collection", "History", "Museum"] },
  { "name": "Librarian and Library Clerk", "keywords": ["Library", "Books", "Information", "Research", "Resources"] },
  { "name": "Editor/Publisher", "keywords": ["Editing", "Publishing", "Content", "Media", "Books"] },
  { "name": "Social Scientist", "keywords": ["Social", "Research", "Society", "Analysis", "Behavior"] },
  { "name": "Archaeologist", "keywords": ["Archaeology", "History", "Excavation", "Artifacts", "Research"] },
  { "name": "Historian", "keywords": ["History", "Research", "Documentation", "Analysis", "Past"] },
  { "name": "Script Writer", "keywords": ["Writing", "Script", "Creative", "Media", "Entertainment"] },
  { "name": "Journalist", "keywords": ["News", "Reporting", "Media", "Writing", "Current Affairs"] },
  { "name": "Language Specialist", "keywords": ["Language", "Linguistics", "Communication", "Translation", "Analysis"] },
  { "name": "Translator and Interpreter", "keywords": ["Translation", "Interpretation", "Languages", "Communication", "International"] },
  { "name": "Director: Film/Stage", "keywords": ["Direction", "Film", "Stage", "Production", "Entertainment"] },
  { "name": "Announcer: Radio, Television and Other media", "keywords": ["Broadcasting", "Media", "Presentation", "Communication", "Entertainment"] },
  { "name": "Health Information Administrator", "keywords": ["Health", "Information", "Administration", "Records", "Healthcare"] },
  { "name": "Museologist", "keywords": ["Museum", "Curator", "Collection", "Exhibition", "Cultural"] },
  { "name": "Secretary", "keywords": ["Administrative", "Office", "Organization", "Support", "Clerical"] },
  { "name": "Broadcasting and Audiovisual Technicians", "keywords": ["Broadcasting", "Technical", "Equipment", "Media", "Audio"] },
  { "name": "Call Centre Operator", "keywords": ["Call Center", "Customer Service", "Communication", "Support", "Telephone"] },
  { "name": "Tour Guide", "keywords": ["Tourism", "Guide", "Travel", "Information", "Cultural"] },
  { "name": "Marketing and Sales Assistant", "keywords": ["Marketing", "Sales", "Business", "Promotion", "Customer"] },
  { "name": "Astronomer", "keywords": ["Astronomy", "Space", "Research", "Stars", "Science"] },
  { "name": "Meteorologist", "keywords": ["Weather", "Climate", "Forecasting", "Atmospheric", "Science"] },
  { "name": "Chemist", "keywords": ["Chemistry", "Laboratory", "Analysis", "Research", "Science"] },
  { "name": "Geologist", "keywords": ["Geology", "Earth", "Rocks", "Minerals", "Science"] },
  { "name": "Geographer", "keywords": ["Geography", "Land", "Spatial", "Maps", "Environment"] },
  { "name": "Mathematician", "keywords": ["Mathematics", "Calculation", "Analysis", "Theory", "Logic"] },
  { "name": "Statistician", "keywords": ["Statistics", "Data", "Analysis", "Probability", "Research"] },
  { "name": "Actuarial Scientist", "keywords": ["Actuarial", "Risk", "Insurance", "Statistics", "Finance"] },
  { "name": "Biologist", "keywords": ["Biology", "Life Science", "Research", "Organisms", "Laboratory"] },
  { "name": "Marine Scientist", "keywords": ["Marine", "Ocean", "Aquatic", "Research", "Biology"] },
  { "name": "Engineer: Biotechnologist", "keywords": ["Biotechnology", "Engineering", "Research", "Biology", "Technology"] },
  { "name": "Biomedical Engineer", "keywords": ["Biomedical", "Engineering", "Healthcare", "Medical", "Technology"] },
  { "name": "Agricultural Scientist", "keywords": ["Agriculture", "Research", "Crops", "Farming", "Science"] },
  { "name": "Engineer: Marine/Fishery Scientists and Aquaculturists", "keywords": ["Marine", "Fishery", "Aquaculture", "Engineering", "Aquatic"] },
  { "name": "Forestry Advisor", "keywords": ["Forestry", "Environment", "Conservation", "Trees", "Management"] },
  { "name": "Environmental Specialist", "keywords": ["Environment", "Conservation", "Ecology", "Sustainability", "Assessment"] },
  { "name": "Industrial Engineer", "keywords": ["Industrial", "Engineering", "Optimization", "Systems", "Efficiency"] },
  { "name": "Food and Drink Technologist", "keywords": ["Food", "Beverage", "Technology", "Processing", "Quality"] },
  { "name": "Civil Engineer", "keywords": ["Civil", "Engineering", "Construction", "Infrastructure", "Design"] },
  { "name": "Automobile Engineer", "keywords": ["Automotive", "Engineering", "Vehicle", "Design", "Mechanical"] },
  { "name": "Mechanical Engineer", "keywords": ["Mechanical", "Engineering", "Design", "Systems", "Machinery"] },
  { "name": "Chemical Engineer", "keywords": ["Chemical", "Engineering", "Process", "Industry", "Materials"] },
  { "name": "Metallurgical Engineer", "keywords": ["Metallurgy", "Engineering", "Metals", "Materials", "Processing"] },
  { "name": "Materials Engineer", "keywords": ["Materials", "Engineering", "Properties", "Development", "Analysis"] },
  { "name": "Electrical Engineer", "keywords": ["Electrical", "Engineering", "Power", "Systems", "Design"] },
  { "name": "Electronics Engineer", "keywords": ["Electronics", "Engineering", "Circuits", "Systems", "Design"] },
  { "name": "Engineer: Telecommunication", "keywords": ["Telecommunication", "Engineering", "Network", "Communication", "Systems"] },
  { "name": "Aerospace/Aeronautical Engineer", "keywords": ["Aerospace", "Aeronautical", "Aircraft", "Engineering", "Flight"] },
  { "name": "Veterinarian", "keywords": ["Veterinary", "Animal", "Medicine", "Healthcare", "Treatment"] },
  { "name": "Veterinarian technician", "keywords": ["Veterinary", "Animal", "Technical", "Assistant", "Healthcare"] },
  { "name": "Pharmacist", "keywords": ["Pharmacy", "Medicine", "Drugs", "Healthcare", "Dispensing"] },
  { "name": "Accountant", "keywords": ["Accounting", "Finance", "Bookkeeping", "Audit", "Financial"] },
  { "name": "Financial Analyst", "keywords": ["Financial", "Analysis", "Investment", "Markets", "Business"] },
  { "name": "Administration Specialist: Oil and Gas", "keywords": ["Administration", "Oil", "Gas", "Management", "Energy"] },
  { "name": "Administration Specialist: Transport", "keywords": ["Administration", "Transport", "Logistics", "Management", "Operations"] },
  { "name": "Administration Specialist: Energy Management", "keywords": ["Administration", "Energy", "Management", "Resources", "Efficiency"] },
  { "name": "Computer Scientist: Software Developer", "keywords": ["Computer", "Software", "Development", "Programming", "Technology"] },
  { "name": "Economist", "keywords": ["Economics", "Markets", "Analysis", "Policy", "Financial"] },
  { "name": "International Trade (Exporter – Importer)", "keywords": ["Trade", "Export", "Import", "International", "Business"] },
  { "name": "Investment Banker", "keywords": ["Investment", "Banking", "Finance", "Markets", "Mergers"] },
  { "name": "Sound Engineer", "keywords": ["Sound", "Audio", "Engineering", "Recording", "Production"] },
  { "name": "Bank Teller and Related Clerks", "keywords": ["Banking", "Teller", "Financial", "Customer Service", "Transactions"] },
  { "name": "Information Technology Teacher", "keywords": ["IT", "Technology", "Teaching", "Education", "Computer"] },
  { "name": "Graphic and Multi Media Designer", "keywords": ["Graphic", "Multimedia", "Design", "Visual", "Digital"] },
  { "name": "Computer database and network administrators", "keywords": ["Database", "Network", "Administration", "IT", "Systems"] },
  { "name": "ICT technician", "keywords": ["ICT", "Technical", "Support", "Computer", "Technology"] },
  { "name": "Treasurer", "keywords": ["Treasury", "Finance", "Management", "Funds", "Financial"] },
  { "name": "Packaging Technologists", "keywords": ["Packaging", "Technology", "Design", "Materials", "Product"] },
  { "name": "Printing Technologists", "keywords": ["Printing", "Technology", "Production", "Press", "Media"] },
  { "name": "Paper Technologists", "keywords": ["Paper", "Technology", "Manufacturing", "Materials", "Production"] },
  { "name": "Garment and Related Trades Technicians", "keywords": ["Garment", "Textile", "Technical", "Fashion", "Production"] },
  { "name": "Leather and Footwear Technologists", "keywords": ["Leather", "Footwear", "Technology", "Manufacturing", "Production"] },
  { "name": "Cosmetologists (Beauticians)", "keywords": ["Cosmetology", "Beauty", "Skincare", "Aesthetics", "Treatment"] },
  { "name": "Architects (Building)", "keywords": ["Architecture", "Building", "Design", "Construction", "Planning"] },
  { "name": "Landscape Designers/Architects", "keywords": ["Landscape", "Design", "Outdoor", "Environment", "Planning"] },
  { "name": "Product/Industrial Designers", "keywords": ["Product", "Design", "Industrial", "Development", "Innovation"] },
  { "name": "Fashion Designers", "keywords": ["Fashion", "Design", "Clothing", "Style", "Apparel"] },
  { "name": "Cartographers and Surveyors", "keywords": ["Cartography", "Surveying", "Maps", "Geographical", "Measurement"] },
  { "name": "Draughts persons", "keywords": ["Drafting", "Technical", "Drawing", "Plans", "Design"] },
  { "name": "Photographers", "keywords": ["Photography", "Visual", "Camera", "Imaging", "Creative"] },
  { "name": "Interior Designers and Decorators", "keywords": ["Interior", "Design", "Decoration", "Space", "Aesthetics"] },
  { "name": "Crafts Designers", "keywords": ["Crafts", "Design", "Artisan", "Handmade", "Creative"] },
  { "name": "Fine Artists", "keywords": ["Art", "Painting", "Sculpture", "Creative", "Artistic"] },
  { "name": "Commercial Artists", "keywords": ["Commercial", "Art", "Design", "Advertising", "Visual"] },
  { "name": "Cinematographers", "keywords": ["Cinematography", "Film", "Camera", "Visual", "Production"] },
  { "name": "Gardeners, Horticultural and Nursery Growers", "keywords": ["Gardening", "Horticulture", "Plants", "Cultivation", "Nursery"] },
  { "name": "Jewelry Designers", "keywords": ["Jewelry", "Design", "Crafts", "Accessories", "Artistic"] },
  { "name": "Potters and Related Workers", "keywords": ["Pottery", "Ceramics", "Crafts", "Artistic", "Production"] },
  { "name": "Sign writers, Decorative Painters and Related Workers", "keywords": ["Sign", "Painting", "Decorative", "Visual", "Artistic"] },
  { "name": "Handicraft Workers in Furniture and Home Furnishings Industries", "keywords": ["Handicraft", "Furniture", "Furnishings", "Artisan", "Production"] },
  { "name": "Handicraft Workers in Textile-Garment and Leather-Footwear Industries", "keywords": ["Handicraft", "Textile", "Garment", "Leather", "Artisan"] },
  { "name": "Machine Operators in Textile-Garment and Leather-Footwear Industries", "keywords": ["Machine", "Operator", "Textile", "Production", "Manufacturing"] },
  { "name": "Allopathic Medical Doctor", "keywords": ["Medical", "Doctor", "Healthcare", "Physician", "Treatment"] },
  { "name": "Nurse and Midwife", "keywords": ["Nursing", "Midwifery", "Healthcare", "Patient", "Medical"] },
  { "name": "Medical Doctor: Traditional Medicine", "keywords": ["Traditional", "Medicine", "Healing", "Alternative", "Healthcare"] },
  { "name": "Dentist", "keywords": ["Dental", "Healthcare", "Oral", "Teeth", "Medical"] },
  { "name": "Occupational Therapist", "keywords": ["Occupational", "Therapy", "Rehabilitation", "Healthcare", "Recovery"] },
  { "name": "Physiotherapist", "keywords": ["Physiotherapy", "Rehabilitation", "Healthcare", "Physical", "Therapy"] },
  { "name": "Dietician and Nutritionist", "keywords": ["Diet", "Nutrition", "Health", "Food", "Wellness"] },
  { "name": "Audiologist and Speech Therapist", "keywords": ["Audiology", "Speech", "Therapy", "Healthcare", "Communication"] },
  { "name": "Pre-School Teacher", "keywords": ["Pre-school", "Teaching", "Early Childhood", "Education", "Children"] },
  { "name": "Primary School Teacher", "keywords": ["Primary", "School", "Teaching", "Education", "Children"] },
  { "name": "Special Needs Teacher", "keywords": ["Special Needs", "Teaching", "Education", "Disability", "Support"] },
  { "name": "Administration Specialist: Small & Medium Business Management", "keywords": ["Administration", "SME", "Management", "Business", "Operations"] },
  { "name": "Agribusiness Management", "keywords": ["Agriculture", "Business", "Management", "Farming", "Commercial"] },
  { "name": "Sales and Marketing Manager", "keywords": ["Sales", "Marketing", "Management", "Business", "Strategy"] },
  { "name": "Retail and Wholesale Manager", "keywords": ["Retail", "Wholesale", "Management", "Commerce", "Business"] },
  { "name": "Hotel/HORECA Manager", "keywords": ["Hotel", "Hospitality", "Management", "Service", "Tourism"] },
  { "name": "Agriculture Manager", "keywords": ["Agriculture", "Management", "Farming", "Production", "Operations"] },
  { "name": "Hospital Administrator", "keywords": ["Hospital", "Administration", "Healthcare", "Management", "Medical"] },
  { "name": "Human Resource Manager", "keywords": ["HR", "Human Resources", "Management", "Personnel", "Recruitment"] },
  { "name": "Administration Specialist: Urban Management", "keywords": ["Urban", "Management", "Administration", "City", "Planning"] },
  { "name": "Labour Relations Officer", "keywords": ["Labor", "Relations", "HR", "Employment", "Industrial"] },
  { "name": "Legal Services Specialist", "keywords": ["Legal", "Services", "Law", "Advisory", "Consultation"] },
  { "name": "Psychologist", "keywords": ["Psychology", "Mental Health", "Therapy", "Behavioral", "Assessment"] },
  { "name": "Counsellor", "keywords": ["Counseling", "Support", "Guidance", "Mental Health", "Therapy"] },
  { "name": "Career Counsellor", "keywords": ["Career", "Counseling", "Guidance", "Employment", "Vocational"] },
  { "name": "Medical Technologist: Medical Imaging", "keywords": ["Medical", "Imaging", "Technology", "Diagnostic", "Radiology"] },
  { "name": "Medical Technologist: Optometry", "keywords": ["Optometry", "Vision", "Technology", "Eye", "Healthcare"] },
  { "name": "Medical Technologist", "keywords": ["Medical", "Technology", "Laboratory", "Healthcare", "Diagnostic"] },
  { "name": "Insurance Agent", "keywords": ["Insurance", "Sales", "Risk", "Policy", "Coverage"] },
  { "name": "Event Manager", "keywords": ["Event", "Management", "Planning", "Coordination", "Organization"] },
  { "name": "Administration Specialist: Real Estate", "keywords": ["Real Estate", "Administration", "Property", "Management", "Commercial"] },
  { "name": "Social Worker", "keywords": ["Social", "Welfare", "Support", "Community", "Assistance"] },
  { "name": "Travel and Tourism Agent", "keywords": ["Travel", "Tourism", "Agency", "Vacation", "Booking"] },
  { "name": "Air Hostess/Steward", "keywords": ["Flight", "Attendant", "Aviation", "Service", "Cabin Crew"] },
  { "name": "Food Processing Workers", "keywords": ["Food", "Processing", "Production", "Preparation", "Manufacturing"] },
  { "name": "Cook", "keywords": ["Cooking", "Culinary", "Food", "Kitchen", "Preparation"] },
  { "name": "Cook Assistant", "keywords": ["Assistant", "Cook", "Kitchen", "Food", "Preparation"] },
  { "name": "Waiter", "keywords": ["Waiting", "Service", "Restaurant", "Food", "Hospitality"] },
  { "name": "Receptionist", "keywords": ["Reception", "Front Desk", "Customer Service", "Administrative", "Office"] },
  { "name": "Office clerk", "keywords": ["Clerical", "Office", "Administrative", "Support", "Filing"] },
  { "name": "Aircraft Maintenance Engineer", "keywords": ["Aircraft", "Maintenance", "Aviation", "Engineering", "Technical"] },
  { "name": "Engineer Mining", "keywords": ["Mining", "Engineering", "Extraction", "Resources", "Geology"] },
  { "name": "Oil and Gas Engineer", "keywords": ["Oil", "Gas", "Engineering", "Petroleum", "Energy"] },
  { "name": "Orthotist and Prosthetist", "keywords": ["Orthotics", "Prosthetics", "Healthcare", "Medical", "Devices"] },
  { "name": "Mechatronics Engineer", "keywords": ["Mechatronics", "Engineering", "Robotics", "Automation", "Systems"] },
  { "name": "Merchant Navy Engineer", "keywords": ["Merchant Navy", "Marine", "Engineering", "Ship", "Maritime"] },
  { "name": "Police Officer", "keywords": ["Police", "Law Enforcement", "Security", "Criminal Justice", "Public Safety"] },
  { "name": "Dancer and Choreographer", "keywords": ["Dance", "Choreography", "Performance", "Artistic", "Creative"] },
  { "name": "Civil Engineering Technician", "keywords": ["Civil", "Engineering", "Technical", "Construction", "Infrastructure"] },
  { "name": "Irrigation technician", "keywords": ["Irrigation", "Water", "Agriculture", "Technical", "Systems"] },
  { "name": "Electrical Engineering Technician (Information-Communication Technology)", "keywords": ["Electrical", "ICT", "Technical", "Engineering", "Communication"] },
  { "name": "Electrical Engineering Technician and Operator (Power Generation and Transmission)", "keywords": ["Electrical", "Power", "Technical", "Generation", "Transmission"] },
  { "name": "Electrical Engineering Technician (Turbines)", "keywords": ["Electrical", "Turbine", "Technical", "Engineering", "Energy"] },
  { "name": "Computer Hardware Technologist", "keywords": ["Computer", "Hardware", "Technology", "Technical", "IT"] },
  { "name": "Ship Radio Officer", "keywords": ["Ship", "Radio", "Maritime", "Communication", "Navigation"] },
  { "name": "Mechanical Engineering Technician (Manufacture, Assembly and Repair)", "keywords": ["Mechanical", "Engineering", "Technical", "Manufacturing", "Repair"] },
  { "name": "Transport Technician", "keywords": ["Transport", "Technical", "Maintenance", "Vehicles", "Logistics"] },
  { "name": "Mining Technician", "keywords": ["Mining", "Technical", "Extraction", "Operations", "Resources"] },
  { "name": "Metallurgical Technician", "keywords": ["Metallurgy", "Technical", "Metals", "Processing", "Materials"] },
  { "name": "Mechanical Engineering Technician: Operator", "keywords": ["Mechanical", "Engineering", "Operator", "Technical", "Machinery"] },
  { "name": "Petroleum and Natural Gas Refining Plant Operator", "keywords": ["Petroleum", "Gas", "Refining", "Operator", "Processing"] },
  { "name": "Environmental Protection Technician", "keywords": ["Environmental", "Protection", "Technical", "Conservation", "Monitoring"] },
  { "name": "Forester", "keywords": ["Forestry", "Conservation", "Trees", "Environmental", "Management"] },
  { "name": "Merchant Navy: Mariner", "keywords": ["Merchant Navy", "Mariner", "Seafaring", "Navigation", "Maritime"] },
  { "name": "Pilot", "keywords": ["Aviation", "Flying", "Aircraft", "Navigation", "Transportation"] },
  { "name": "Air Traffic Controller", "keywords": ["Air Traffic", "Control", "Aviation", "Safety", "Navigation"] },
  { "name": "Medical Technologist: Nuclear Medicine Technology", "keywords": ["Nuclear", "Medicine", "Medical", "Imaging", "Radiological"] },
  { "name": "Laboratory Technician", "keywords": ["Laboratory", "Technical", "Analysis", "Testing", "Research"] },
  { "name": "Medical Technologist: Surgical Technology", "keywords": ["Surgical", "Technology", "Medical", "Operating Room", "Healthcare"] },
  { "name": "Dental Assistant and Therapist", "keywords": ["Dental", "Assistant", "Oral", "Healthcare", "Support"] },
  { "name": "Sports Professional", "keywords": ["Sports", "Athletic", "Professional", "Physical", "Competition"] },
  { "name": "Physical Education Teacher", "keywords": ["Physical", "Education", "Teaching", "Sports", "Fitness"] },
  { "name": "Telecommunications Technician", "keywords": ["Telecommunications", "Technical", "Network", "Communication", "Installation"] },
  { "name": "Fire-Fighter", "keywords": ["Fire", "Emergency", "Rescue", "Safety", "Response"] },
  { "name": "Crop Grower", "keywords": ["Crops", "Agriculture", "Farming", "Cultivation", "Production"] },
  { "name": "Animal Husbandry Specialist", "keywords": ["Animal", "Husbandry", "Livestock", "Breeding", "Farming"] },
  { "name": "Apiarist and Sericulturist", "keywords": ["Beekeeping", "Sericulture", "Agriculture", "Insects", "Production"] },
  { "name": "Plumber and Pipe Fitter", "keywords": ["Plumbing", "Pipefitting", "Installation", "Maintenance", "Construction"] },
  { "name": "Tool and Die Maker", "keywords": ["Tool", "Die", "Manufacturing", "Precision", "Metalwork"] },
  { "name": "Carpenter and Joiner", "keywords": ["Carpentry", "Woodworking", "Construction", "Joinery", "Building"] },
  { "name": "HVAC Technician", "keywords": ["HVAC", "Heating", "Cooling", "Ventilation", "Technical"] },
  { "name": "Welder and Flame Cutter", "keywords": ["Welding", "Cutting", "Metal", "Fabrication", "Construction"] },
  { "name": "Mechanical Engineering Technician (Ship Building)", "keywords": ["Mechanical", "Shipbuilding", "Technical", "Marine", "Construction"] },
  { "name": "Mechanical Technician (Metal Working)", "keywords": ["Mechanical", "Metalworking", "Technical", "Fabrication", "Manufacturing"] },
  { "name": "Auto Mechanic", "keywords": ["Automotive", "Mechanic", "Repair", "Vehicle", "Maintenance"] },
  { "name": "Mechanical Engineering Technician: Agricultural", "keywords": ["Mechanical", "Agricultural", "Technical", "Farm", "Equipment"] },
  { "name": "Underwater Diver", "keywords": ["Diving", "Underwater", "Marine", "Commercial", "Exploration"] },
  { "name": "Locomotive Driver", "keywords": ["Locomotive", "Train", "Railway", "Transportation", "Operator"] },
  { "name": "Car, Taxi and Light Truck Driver", "keywords": ["Driver", "Taxi", "Transportation", "Vehicle", "Service"] },
  { "name": "Bus and Tram Driver", "keywords": ["Bus", "Tram", "Driver", "Public Transport", "Transportation"] },
  { "name": "Army officer", "keywords": ["Army", "Military", "Officer", "Defense", "Leadership"] },
  { "name": "Forestry Worker", "keywords": ["Forestry", "Trees", "Logging", "Conservation", "Outdoor"] },
  { "name": "Aquaculture and Fishery Worker", "keywords": ["Aquaculture", "Fishery", "Marine", "Fish", "Production"] },
  { "name": "Mason", "keywords": ["Masonry", "Construction", "Building", "Stonework", "Brickwork"] },
  { "name": "Building Finisher", "keywords": ["Building", "Finishing", "Construction", "Interior", "Craftsmanship"] },
  { "name": "Painter", "keywords": ["Painting", "Construction", "Finishing", "Decorating", "Surface"] },
  { "name": "Electrical Installation and Repair Worker", "keywords": ["Electrical", "Installation", "Repair", "Wiring", "Maintenance"] },
  { "name": "Electronics and Telecommunications Installation and Repair Worker", "keywords": ["Electronics", "Telecommunications", "Installation", "Repair", "Systems"] },
  { "name": "Assembler", "keywords": ["Assembly", "Manufacturing", "Production", "Components", "Factory"] }
]

# Hàm chuẩn hóa tiêu đề
def normalize_title(title):
    if title is None:
        return "Unknown"  # Giá trị mặc định từ danh sách chuẩn
    
    # Chuẩn hóa title trước khi tìm kiếm trong mapping
    normalized = re.sub(r'\([^)]*\)', '', title)  # Xóa nội dung trong ngoặc ()
    normalized = re.sub(r'-.*$', '', normalized)   # Xóa nội dung sau dấu gạch ngang
    normalized = normalized.strip()
    
    # Tìm kiếm chính xác trong mapping
    if normalized in title_mapping:
        return title_mapping[normalized]
    
    # Tìm các từ khóa chính trong title
    for key, value in title_mapping.items():
        if key.lower() in normalized.lower():
            return value
    
    # Nếu không tìm thấy, trả về một giá trị mặc định từ danh sách
    return "Unknown"  # Hoặc một giá trị khác từ danh sách chuẩn

# Đăng ký UDF
normalize_title_udf = udf(normalize_title, StringType())

# Áp dụng UDF để chuẩn hóa tiêu đề
job_data_normalized = job_data.withColumn('job_title', normalize_title_udf(job_data['job_title']))

# Sử dụng pandas để tránh lỗi Hadoop
pandas_df = job_data_normalized.toPandas()
pandas_df.to_csv('jobs_data_careerviet2_final_normalized.csv', index=False)

print("Đã hoàn thành việc chuẩn hóa tiêu đề và lưu vào file 'jobs_data_careerviet2_final_normalized.csv'.")

# Dừng SparkSession
spark.stop()

Py4JJavaError: An error occurred while calling o66.collectToPython.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 1.0 failed 1 times, most recent failure: Lost task 0.0 in stage 1.0 (TID 1) (LENOVO executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.sql.execution.python.BatchEvalPythonExec.evaluate(BatchEvalPythonExec.scala:54)
	at org.apache.spark.sql.execution.python.EvalPythonExec.$anonfun$doExecute$2(EvalPythonExec.scala:131)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitions$2(RDD.scala:858)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartition

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
import re

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("Normalize Job Titles") \
    .config("spark.driver.memory", "4g") \
    .getOrCreate()

# Đọc file CSV với tham số xử lý lỗi
job_data = spark.read.option("header", "true") \
    .option("mode", "PERMISSIVE") \
    .option("multiLine", "true") \
    .option("escape", "\\") \
    .option("quote", "\"") \
    .csv('data/jobs_data_careerviet2_final.csv')

# Tạo mapping mới với key là tiêu đề tiếng Anh và value là mảng các từ khóa tiếng Việt (giữ nguyên dấu)
professions = [
    {"name": "University Teacher", "keywords": ["Giáo sư", "Đại học", "Giảng dạy", "Học thuật", "Giáo dục", "Chuyên viên", ]},
  {"name": "College Teacher", "keywords": ["Giảng viên", "Cao đẳng", "Giáo dục", "Đào tạo", "Giảng dạy"]},
  {"name": "Secondary Vocational and Professional School Teacher", "keywords": ["Dạy nghề", "Chuyên nghiệp", "Giáo viên", "Đào tạo", "Kỹ năng"]},
  {"name": "Upper Secondary Education Teacher", "keywords": ["Trung học phổ thông", "Giáo viên", "Giáo dục", "Trung học", "Chương trình"]},
  {"name": "Lower Secondary Education Teacher", "keywords": ["Trung học cơ sở", "Giáo viên", "Giáo dục", "Thanh thiếu niên", "Dạy học"]},
  {"name": "Foreign Language Teacher", "keywords": ["Ngoại ngữ", "Giáo viên", "Ngôn ngữ", "Quốc tế", "Giao tiếp"]},
  {"name": "Public Relations Officer", "keywords": ["Quan hệ công chúng", "Truyền thông", "PR", "Doanh nghiệp", "Hình ảnh"]},
  {"name": "Lawyer", "keywords": ["Luật sư", "Pháp luật", "Tư vấn", "Biện hộ", "Tư pháp"]},
  {"name": "Archivist and Curator", "keywords": ["Lưu trữ", "Quản lý bảo tàng", "Bảo quản", "Hiện vật", "Lịch sử"]},
  {"name": "Librarian and Library Clerk", "keywords": ["Thủ thư", "Thư viện", "Sách", "Tài liệu", "Quản lý thông tin"]},
  {"name": "Editor/Publisher", "keywords": ["Biên tập viên", "Xuất bản", "Báo chí", "Nội dung", "Sách"]},
  {"name": "Social Scientist", "keywords": ["Nhà khoa học xã hội", "Nghiên cứu", "Xã hội học", "Phân tích", "Hành vi"]},
  {"name": "Archaeologist", "keywords": ["Nhà khảo cổ học", "Khai quật", "Di tích", "Lịch sử", "Cổ vật"]},
  {"name": "Historian", "keywords": ["Sử học", "Nghiên cứu lịch sử", "Tài liệu", "Phân tích", "Quá khứ"]},
  {"name": "Script Writer", "keywords": ["Biên kịch", "Kịch bản", "Sáng tạo", "Phim ảnh", "Truyền thông"]},
  {"name": "Journalist", "keywords": ["Nhà báo", "Tin tức", "Phóng viên", "Truyền thông", "Thời sự"]},
  {"name": "Language Specialist", "keywords": ["Chuyên gia ngôn ngữ", "Ngôn ngữ học", "Giao tiếp", "Phiên dịch", "Phân tích"]},
  {"name": "Translator and Interpreter", "keywords": ["Biên dịch", "Phiên dịch", "Ngôn ngữ", "Giao tiếp", "Quốc tế"]},
  {"name": "Director: Film/Stage", "keywords": ["Đạo diễn", "Phim", "Sân khấu", "Sản xuất", "Giải trí"]},
  {"name": "Announcer: Radio, Television and Other media", "keywords": ["Người dẫn chương trình", "Phát thanh", "Truyền hình", "Truyền thông", "MC"]},
  {"name": "Health Information Administrator", "keywords": ["Quản lý thông tin y tế", "Y tế", "Hồ sơ", "Quản lý", "Dữ liệu"]},
  {"name": "Museologist", "keywords": ["Chuyên gia bảo tàng", "Triển lãm", "Bảo tàng", "Di sản", "Văn hóa"]},
  {"name": "Secretary", "keywords": ["Thư ký", "Hành chính", "Văn phòng", "Trợ lý", "Điều hành"]},
  {"name": "Broadcasting and Audiovisual Technicians", "keywords": ["Kỹ thuật viên phát thanh", "Kỹ thuật viên nghe nhìn", "Truyền hình", "Thiết bị", "Kỹ thuật"]},
  {"name": "Call Centre Operator", "keywords": ["Tổng đài viên", "Chăm sóc khách hàng", "Điện thoại", "Hỗ trợ", "Liên lạc"]},
  {"name": "Tour Guide", "keywords": ["Hướng dẫn viên du lịch", "Du lịch", "Lữ hành", "Văn hóa", "Thông tin"]},
  {"name": "Marketing and Sales Assistant", "keywords": ["Trợ lý tiếp thị", "Trợ lý bán hàng", "Marketing", "Kinh doanh", "Khách hàng"]},
  {"name": "Astronomer", "keywords": ["Nhà thiên văn học", "Vũ trụ", "Thiên văn", "Nghiên cứu", "Khoa học"]},
  {"name": "Meteorologist", "keywords": ["Nhà khí tượng học", "Thời tiết", "Khí hậu", "Dự báo", "Khoa học"]},
  {"name": "Chemist", "keywords": ["Nhà hóa học", "Hóa chất", "Phòng thí nghiệm", "Nghiên cứu", "Phân tích"]},
  {"name": "Geologist", "keywords": ["Nhà địa chất học", "Địa chất", "Khoáng sản", "Đất đá", "Khoa học"]},
  {"name": "Geographer", "keywords": ["Nhà địa lý học", "Bản đồ", "Không gian", "Môi trường", "Địa hình"]},
  {"name": "Mathematician", "keywords": ["Nhà toán học", "Toán học", "Tính toán", "Lý thuyết", "Logic"]},
  {"name": "Statistician", "keywords": ["Nhà thống kê", "Dữ liệu", "Phân tích", "Xác suất", "Nghiên cứu"]},
  {"name": "Actuarial Scientist", "keywords": ["Chuyên gia tính toán bảo hiểm", "Phân tích rủi ro", "Bảo hiểm", "Thống kê", "Tài chính"]},
  {"name": "Biologist", "keywords": ["Nhà sinh học", "Sinh vật", "Nghiên cứu", "Khoa học", "Phòng thí nghiệm"]},
  {"name": "Marine Scientist", "keywords": ["Nhà khoa học biển", "Đại dương", "Biển", "Sinh vật biển", "Nghiên cứu"]},
  {"name": "Engineer: Biotechnologist", "keywords": ["Kỹ sư công nghệ sinh học", "Sinh học", "Công nghệ", "Nghiên cứu", "Kỹ thuật"]},
  {"name": "Biomedical Engineer", "keywords": ["Kỹ sư y sinh", "Y tế", "Thiết bị y tế", "Công nghệ", "Y học"]},
  {"name": "Agricultural Scientist", "keywords": ["Nhà khoa học nông nghiệp", "Canh tác", "Nông nghiệp", "Cây trồng", "Nghiên cứu"]},
  {"name": "Engineer: Marine/Fishery Scientists and Aquaculturists", "keywords": ["Kỹ sư thủy sản", "Ngư nghiệp", "Nuôi trồng thủy sản", "Biển", "Môi trường"]},
  {"name": "Forestry Advisor", "keywords": ["Cố vấn lâm nghiệp", "Rừng", "Bảo tồn", "Môi trường", "Quản lý"]},
  {"name": "Environmental Specialist", "keywords": ["Chuyên gia môi trường", "Bảo vệ môi trường", "Sinh thái", "Bền vững", "Đánh giá"]},
  {"name": "Industrial Engineer", "keywords": ["Kỹ sư công nghiệp", "Tối ưu hóa", "Hệ thống", "Hiệu quả", "Sản xuất"]},
  {"name": "Food and Drink Technologist", "keywords": ["Kỹ thuật viên thực phẩm đồ uống", "Chế biến", "Công nghệ", "Chất lượng", "Sản xuất"]},
  {"name": "Civil Engineer", "keywords": ["Kỹ sư xây dựng", "Công trình", "Cơ sở hạ tầng", "Thiết kế", "Xây dựng"]},
  {"name": "Automobile Engineer", "keywords": ["Kỹ sư ô tô", "Xe hơi", "Thiết kế", "Cơ khí", "Phương tiện"]},
  {"name": "Mechanical Engineer", "keywords": ["Kỹ sư cơ khí", "Máy móc", "Thiết kế", "Hệ thống", "Cơ khí"]},
  {"name": "Chemical Engineer", "keywords": ["Kỹ sư hóa học", "Hóa chất", "Quy trình", "Công nghiệp", "Vật liệu"]},
  {"name": "Metallurgical Engineer", "keywords": ["Kỹ sư luyện kim", "Kim loại", "Vật liệu", "Chế biến", "Công nghiệp"]},
  {"name": "Materials Engineer", "keywords": ["Kỹ sư vật liệu", "Chất liệu", "Phát triển", "Nghiên cứu", "Tính chất"]},
  {"name": "Electrical Engineer", "keywords": ["Kỹ sư điện", "Điện lực", "Hệ thống", "Thiết kế", "Năng lượng"]},
  {"name": "Electronics Engineer", "keywords": ["Kỹ sư điện tử", "Mạch điện", "Hệ thống", "Thiết kế", "Điện tử"]},
  {"name": "Engineer: Telecommunication", "keywords": ["Kỹ sư viễn thông", "Mạng lưới", "Truyền thông", "Hệ thống", "Kết nối"]},
  {"name": "Aerospace/Aeronautical Engineer", "keywords": ["Kỹ sư hàng không vũ trụ", "Máy bay", "Kỹ thuật", "Thiết kế", "Bay"]},
  {"name": "Veterinarian", "keywords": ["Bác sĩ thú y", "Động vật", "Y tế", "Điều trị", "Chăm sóc"]},
  {"name": "Veterinarian technician", "keywords": ["Kỹ thuật viên thú y", "Động vật", "Hỗ trợ", "Chăm sóc", "Y tế"]},
  {"name": "Pharmacist", "keywords": ["Dược sĩ", "Thuốc", "Y tế", "Dược phẩm", "Điều chế"]},
  {"name": "Accountant", "keywords": ["Kế toán viên", "Tài chính", "Sổ sách", "Kiểm toán", "Kế toán"]},
  {"name": "Financial Analyst", "keywords": ["Chuyên viên phân tích tài chính", "Đầu tư", "Thị trường", "Tài chính", "Kinh doanh"]},
  {"name": "Administration Specialist: Oil and Gas", "keywords": ["Chuyên viên quản lý dầu khí", "Dầu mỏ", "Khí đốt", "Quản lý", "Năng lượng"]},
  {"name": "Administration Specialist: Transport", "keywords": ["Chuyên viên quản lý vận tải", "Vận chuyển", "Logistics", "Quản lý", "Vận hành"]},
  {"name": "Administration Specialist: Energy Management", "keywords": ["Chuyên viên quản lý năng lượng", "Năng lượng", "Hiệu quả", "Quản lý", "Tài nguyên"]},
  {"name": "Computer Scientist: Software Developer", "keywords": ["Lập trình viên", "Phần mềm", "Công nghệ", "Lập trình", "IT"]},
  {"name": "Economist", "keywords": ["Nhà kinh tế học", "Kinh tế", "Thị trường", "Phân tích", "Chính sách"]},
  {"name": "International Trade (Exporter – Importer)", "keywords": ["Thương mại quốc tế", "Xuất khẩu", "Nhập khẩu", "Kinh doanh", "Quốc tế"]},
  {"name": "Investment Banker", "keywords": ["Chuyên viên ngân hàng đầu tư", "Đầu tư", "Tài chính", "Thị trường", "Sáp nhập"]},
  {"name": "Sound Engineer", "keywords": ["Kỹ sư âm thanh", "Âm nhạc", "Ghi âm", "Sản xuất", "Thiết bị"]},
  {"name": "Bank Teller and Related Clerks", "keywords": ["Giao dịch viên ngân hàng", "Tài chính", "Giao dịch", "Khách hàng", "Ngân hàng"]},
  {"name": "Information Technology Teacher", "keywords": ["Giáo viên công nghệ thông tin", "IT", "Giảng dạy", "Máy tính", "Giáo dục"]},
  {"name": "Graphic and Multi Media Designer", "keywords": ["Nhà thiết kế đồ họa", "Thiết kế", "Đa phương tiện", "Hình ảnh", "Sáng tạo"]},
  {"name": "Computer database and network administrators", "keywords": ["Quản trị mạng", "Cơ sở dữ liệu", "Hệ thống", "IT", "Quản lý"]},
  {"name": "ICT technician", "keywords": ["Kỹ thuật viên CNTT", "Hỗ trợ", "Máy tính", "Công nghệ", "Kỹ thuật"]},
  {"name": "Treasurer", "keywords": ["Thủ quỹ", "Tài chính", "Quản lý", "Ngân sách", "Quỹ"]},
  {"name": "Packaging Technologists", "keywords": ["Chuyên gia đóng gói", "Bao bì", "Thiết kế", "Sản phẩm", "Công nghệ"]},
  {"name": "Printing Technologists", "keywords": ["Chuyên gia in ấn", "In", "Sản xuất", "Công nghệ", "Ấn phẩm"]},
  {"name": "Paper Technologists", "keywords": ["Chuyên gia công nghệ giấy", "Giấy", "Sản xuất", "Vật liệu", "Công nghệ"]},
  {"name": "Garment and Related Trades Technicians", "keywords": ["Kỹ thuật viên may mặc", "Dệt may", "Thời trang", "Sản xuất", "Kỹ thuật"]},
  {"name": "Leather and Footwear Technologists", "keywords": ["Chuyên gia da giày", "Da", "Giày dép", "Sản xuất", "Công nghệ"]},
  {"name": "Cosmetologists (Beauticians)", "keywords": ["Chuyên viên làm đẹp", "Thẩm mỹ", "Chăm sóc da", "Làm đẹp", "Mỹ phẩm"]},
  {"name": "Architects (Building)", "keywords": ["Kiến trúc sư", "Thiết kế", "Xây dựng", "Công trình", "Quy hoạch"]},
  {"name": "Landscape Designers/Architects", "keywords": ["Kiến trúc sư cảnh quan", "Thiết kế", "Môi trường", "Ngoại thất", "Cảnh quan"]},
  {"name": "Product/Industrial Designers", "keywords": ["Nhà thiết kế sản phẩm", "Thiết kế công nghiệp", "Sản phẩm", "Sáng tạo", "Phát triển"]},
  {"name": "Fashion Designers", "keywords": ["Nhà thiết kế thời trang", "Thời trang", "Trang phục", "Thiết kế", "Xu hướng"]},
  {"name": "Cartographers and Surveyors", "keywords": ["Nhà đo đạc bản đồ", "Khảo sát", "Bản đồ", "Địa lý", "Đo lường"]},
  {"name": "Draughts persons", "keywords": ["Họa viên kỹ thuật", "Vẽ kỹ thuật", "Bản vẽ", "Kỹ thuật", "Thiết kế"]},
  {"name": "Photographers", "keywords": ["Nhiếp ảnh gia", "Chụp ảnh", "Hình ảnh", "Máy ảnh", "Sáng tạo"]},
  {"name": "Interior Designers and Decorators", "keywords": ["Nhà thiết kế nội thất", "Trang trí", "Không gian", "Nội thất", "Thẩm mỹ"]},
  {"name": "Crafts Designers", "keywords": ["Nhà thiết kế thủ công", "Thủ công", "Nghệ thuật", "Sáng tạo", "Mỹ nghệ"]},
  {"name": "Fine Artists", "keywords": ["Họa sĩ", "Nghệ thuật", "Hội họa", "Điêu khắc", "Sáng tạo"]},
  {"name": "Commercial Artists", "keywords": ["Họa sĩ thương mại", "Quảng cáo", "Mỹ thuật", "Thiết kế", "Thương mại"]},
  {"name": "Cinematographers", "keywords": ["Quay phim", "Điện ảnh", "Máy quay", "Hình ảnh", "Sản xuất"]},
  {"name": "Gardeners, Horticultural and Nursery Growers", "keywords": ["Nhà làm vườn", "Làm vườn", "Cây trồng", "Trồng trọt", "Cây cảnh"]},
  {"name": "Jewelry Designers", "keywords": ["Nhà thiết kế trang sức", "Trang sức", "Mỹ nghệ", "Thiết kế", "Thủ công"]},
  {"name": "Potters and Related Workers", "keywords": ["Thợ gốm", "Gốm sứ", "Thủ công", "Nghệ thuật", "Sản xuất"]},
  {"name": "Sign writers, Decorative Painters and Related Workers", "keywords": ["Thợ vẽ biển hiệu", "Vẽ trang trí", "Mỹ thuật", "Biển hiệu", "Nghệ thuật"]},
  {"name": "Handicraft Workers in Furniture and Home Furnishings Industries", "keywords": ["Thợ thủ công nội thất", "Đồ gỗ", "Nội thất", "Thủ công", "Sản xuất"]},
  {"name": "Handicraft Workers in Textile-Garment and Leather-Footwear Industries", "keywords": ["Thợ thủ công dệt may", "Da giày", "Thủ công", "Dệt may", "Sản xuất"]},
  {"name": "Machine Operators in Textile-Garment and Leather-Footwear Industries", "keywords": ["Công nhân vận hành máy dệt may", "Da giày", "Sản xuất", "Máy móc", "Nhà máy"]},
  {"name": "Allopathic Medical Doctor", "keywords": ["Bác sĩ", "Y khoa", "Y tế", "Khám chữa bệnh", "Điều trị"]},
  {"name": "Nurse and Midwife", "keywords": ["Y tá", "Hộ sinh", "Điều dưỡng", "Y tế", "Chăm sóc"]},
  {"name": "Medical Doctor: Traditional Medicine", "keywords": ["Bác sĩ y học cổ truyền", "Đông y", "Y học", "Chữa bệnh", "Truyền thống"]},
  {"name": "Dentist", "keywords": ["Nha sĩ", "Răng", "Nha khoa", "Y tế", "Điều trị"]},
  {"name": "Occupational Therapist", "keywords": ["Nhà trị liệu hoạt động", "Phục hồi", "Chức năng", "Y tế", "Điều trị"]},
  {"name": "Physiotherapist", "keywords": ["Nhà vật lý trị liệu", "Phục hồi", "Chức năng", "Điều trị", "Phục hồi"]},
  {"name": "Dietician and Nutritionist", "keywords": ["Chuyên gia dinh dưỡng", "Dinh dưỡng", "Thực phẩm", "Sức khỏe", "Chế độ ăn"]},
  {"name": "Audiologist and Speech Therapist", "keywords": ["Chuyên gia thính học", "Trị liệu ngôn ngữ", "Nghe", "Nói", "Y tế"]},
  {"name": "Pre-School Teacher", "keywords": ["Giáo viên mầm non", "Trẻ em", "Giáo dục", "Mầm non", "Dạy học"]},
  {"name": "Primary School Teacher", "keywords": ["Giáo viên tiểu học", "Tiểu học", "Giáo dục", "Trẻ em", "Dạy học"]},
  {"name": "Special Needs Teacher", "keywords": ["Giáo viên giáo dục đặc biệt", "Đặc biệt", "Khuyết tật", "Giáo dục", "Hỗ trợ"]},
  {"name": "Administration Specialist: Small & Medium Business Management", "keywords": ["Chuyên viên quản lý doanh nghiệp vừa và nhỏ", "SME", "Quản lý", "Kinh doanh", "Vận hành"]},
  {"name": "Agribusiness Management", "keywords": ["Quản lý nông nghiệp", "Nông nghiệp", "Kinh doanh", "Quản lý", "Thương mại"]},
  {"name": "Sales and Marketing Manager", "keywords": ["Giám đốc kinh doanh tiếp thị", "Marketing", "Bán hàng", "Chiến lược", "Quản lý"]},
  {"name": "Retail and Wholesale Manager", "keywords": ["Quản lý bán lẻ bán sỉ", "Bán lẻ", "Bán sỉ", "Thương mại", "Quản lý"]},
  {"name": "Hotel/HORECA Manager", "keywords": ["Quản lý khách sạn nhà hàng", "Khách sạn", "Dịch vụ", "Du lịch", "Quản lý"]},
  {"name": "Agriculture Manager", "keywords": ["Quản lý nông nghiệp", "Nông nghiệp", "Canh tác", "Sản xuất", "Quản lý"]},
  {"name": "Hospital Administrator", "keywords": ["Quản lý bệnh viện", "Y tế", "Bệnh viện", "Quản lý", "Điều hành"]},
  {"name": "Human Resource Manager", "keywords": ["Giám đốc nhân sự", "Nhân sự", "Tuyển dụng", "Quản lý", "Nhân viên"]},
  {"name": "Administration Specialist: Urban Management", "keywords": ["Chuyên viên quản lý đô thị", "Đô thị", "Quy hoạch", "Quản lý", "Thành phố"]},
  {"name": "Labour Relations Officer", "keywords": ["Chuyên viên quan hệ lao động", "Lao động", "Nhân sự", "Công đoàn", "Quan hệ"]},
  {"name": "Legal Services Specialist", "keywords": ["Chuyên viên dịch vụ pháp lý", "Pháp luật", "Tư vấn", "Dịch vụ", "Luật"]},
  {"name": "Psychologist", "keywords": ["Nhà tâm lý học", "Tâm lý", "Sức khỏe tâm thần", "Trị liệu", "Đánh giá"]},
  {"name": "Counsellor", "keywords": ["Nhà tư vấn", "Tư vấn", "Hỗ trợ", "Tâm lý", "Trị liệu"]},
  {"name": "Career Counsellor", "keywords": ["Cố vấn nghề nghiệp", "Hướng nghiệp", "Tư vấn", "Nghề nghiệp", "Việc làm"]},
  {"name": "Medical Technologist: Medical Imaging", "keywords": ["Kỹ thuật viên chẩn đoán hình ảnh", "Y tế", "Chẩn đoán", "Hình ảnh", "X-quang"]},
  {"name": "Medical Technologist: Optometry", "keywords": ["Kỹ thuật viên nhãn khoa", "Mắt", "Thị lực", "Y tế", "Nhãn khoa"]},
  {"name": "Medical Technologist", "keywords": ["Kỹ thuật viên y tế", "Y tế", "Xét nghiệm", "Chẩn đoán", "Kỹ thuật"]},
  {"name": "Insurance Agent", "keywords": ["Đại lý bảo hiểm", "Bảo hiểm", "Tư vấn", "Hợp đồng", "Dịch vụ"]},
  {"name": "Event Manager", "keywords": ["Quản lý sự kiện", "Sự kiện", "Tổ chức", "Điều phối", "Kế hoạch"]},
  {"name": "Administration Specialist: Real Estate", "keywords": ["Chuyên viên quản lý bất động sản", "Bất động sản", "Địa ốc", "Quản lý", "Tài sản"]},
  {"name": "Social Worker", "keywords": ["Nhân viên xã hội", "Xã hội", "Hỗ trợ", "Cộng đồng", "Phúc lợi"]},
  {"name": "Travel and Tourism Agent", "keywords": ["Đại lý du lịch", "Du lịch", "Lữ hành", "Đặt tour", "Dịch vụ"]},
  {"name": "Air Hostess/Steward", "keywords": ["Tiếp viên hàng không", "Máy bay", "Phục vụ", "Hàng không", "Dịch vụ"]},
  {"name": "Food Processing Workers", "keywords": ["Công nhân chế biến thực phẩm", "Thực phẩm", "Chế biến", "Sản xuất", "Nhà máy"]},
  {"name": "Cook", "keywords": ["Đầu bếp", "Nấu ăn", "Ẩm thực", "Nhà hàng", "Chế biến"]},
  {"name": "Cook Assistant", "keywords": ["Phụ bếp", "Hỗ trợ", "Nhà bếp", "Chế biến", "Nhà hàng"]},
  {"name": "Waiter", "keywords": ["Bồi bàn", "Phục vụ", "Nhà hàng", "Dịch vụ", "Khách hàng"]},
  {"name": "Receptionist", "keywords": ["Lễ tân", "Tiếp tân", "Đón tiếp", "Văn phòng", "Khách hàng"]},
  {"name": "Office clerk", "keywords": ["Nhân viên văn phòng", "Hành chính", "Văn thư", "Văn phòng", "Giấy tờ"]},
  {"name": "Aircraft Maintenance Engineer", "keywords": ["Kỹ sư bảo trì máy bay", "Hàng không", "Bảo dưỡng", "Kỹ thuật", "Máy bay"]},
  {"name": "Engineer Mining", "keywords": ["Kỹ sư khai thác mỏ", "Khai thác", "Mỏ", "Địa chất", "Tài nguyên"]},
  {"name": "Oil and Gas Engineer", "keywords": ["Kỹ sư dầu khí", "Dầu mỏ", "Khí đốt", "Năng lượng", "Khai thác"]},
  {"name": "Orthotist and Prosthetist", "keywords": ["Chuyên gia chỉnh hình", "Bộ phận giả", "Y tế", "Phục hồi", "Thiết bị"]},
  {"name": "Mechatronics Engineer", "keywords": ["Kỹ sư cơ điện tử", "Tự động hóa", "Robot", "Hệ thống", "Điều khiển"]},
  {"name": "Merchant Navy Engineer", "keywords": ["Kỹ sư hàng hải", "Tàu biển", "Hàng hải", "Vận hành", "Kỹ thuật"]},
  {"name": "Police Officer", "keywords": ["Cảnh sát", "An ninh", "Pháp luật", "Trật tự", "Hình sự"]},
  {"name": "Dancer and Choreographer", "keywords": ["Vũ công", "Biên đạo múa", "Nghệ thuật", "Nhảy múa", "Biểu diễn"]},
  {"name": "Civil Engineering Technician", "keywords": ["Kỹ thuật viên xây dựng", "Công trình", "Xây dựng", "Kỹ thuật", "Hạ tầng"]},
  {"name": "Irrigation technician", "keywords": ["Kỹ thuật viên thủy lợi", "Tưới tiêu", "Nông nghiệp", "Thủy lợi", "Hệ thống"]},
  {"name": "Electrical Engineering Technician (Information-Communication Technology)", "keywords": ["Kỹ thuật viên điện ICT", "Công nghệ thông tin", "Điện tử", "Truyền thông", "Kỹ thuật"]},
  {"name": "Electrical Engineering Technician and Operator (Power Generation and Transmission)", "keywords": ["Kỹ thuật viên điện lực", "Phát điện", "Truyền tải", "Điện", "Vận hành"]},
  {"name": "Electrical Engineering Technician (Turbines)", "keywords": ["Kỹ thuật viên tuabin điện", "Tuabin", "Điện", "Năng lượng", "Kỹ thuật"]},
  {"name": "Computer Hardware Technologist", "keywords": ["Chuyên gia phần cứng máy tính", "Phần cứng", "Máy tính", "Thiết bị", "Công nghệ"]},
  {"name": "Ship Radio Officer", "keywords": ["Sĩ quan vô tuyến tàu biển", "Thông tin", "Tàu biển", "Vô tuyến", "Hàng hải"]},
  {"name": "Mechanical Engineering Technician (Manufacture, Assembly and Repair)", "keywords": ["Kỹ thuật viên cơ khí", "Sản xuất", "Lắp ráp", "Sửa chữa", "Máy móc"]},
  {"name": "Transport Technician", "keywords": ["Kỹ thuật viên vận tải", "Vận tải", "Phương tiện", "Bảo trì", "Kỹ thuật"]},
  {"name": "Mining Technician", "keywords": ["Kỹ thuật viên khai thác mỏ", "Khai thác", "Mỏ", "Hoạt động", "Tài nguyên"]},
  {"name": "Metallurgical Technician", "keywords": ["Kỹ thuật viên luyện kim", "Kim loại", "Vật liệu", "Xử lý", "Kỹ thuật"]},
  {"name": "Mechanical Engineering Technician: Operator", "keywords": ["Kỹ thuật viên vận hành cơ khí", "Vận hành", "Máy móc", "Cơ khí", "Thiết bị"]},
  {"name": "Petroleum and Natural Gas Refining Plant Operator", "keywords": ["Người vận hành nhà máy lọc dầu khí", "Dầu mỏ", "Khí đốt", "Lọc", "Vận hành"]},
  {"name": "Environmental Protection Technician", "keywords": ["Kỹ thuật viên bảo vệ môi trường", "Môi trường", "Bảo vệ", "Giám sát", "Sinh thái"]},
  {"name": "Forester", "keywords": ["Kiểm lâm", "Rừng", "Bảo tồn", "Quản lý", "Môi trường"]},
  {"name": "Merchant Navy: Mariner", "keywords": ["Thủy thủ hàng hải", "Tàu biển", "Hàng hải", "Đi biển", "Vận chuyển"]},
  {"name": "Pilot", "keywords": ["Phi công", "Lái máy bay", "Hàng không", "Bay", "Vận tải"]},
  {"name": "Air Traffic Controller", "keywords": ["Kiểm soát viên không lưu", "Không lưu", "Hàng không", "An toàn", "Điều hành"]},
  {"name": "Medical Technologist: Nuclear Medicine Technology", "keywords": ["Kỹ thuật viên y học hạt nhân", "Hạt nhân", "Y tế", "Chẩn đoán", "Xạ"]},
  {"name": "Laboratory Technician", "keywords": ["Kỹ thuật viên phòng thí nghiệm", "Xét nghiệm", "Phân tích", "Nghiên cứu", "Thí nghiệm"]},
  {"name": "Medical Technologist: Surgical Technology", "keywords": ["Kỹ thuật viên phẫu thuật", "Phẫu thuật", "Y tế", "Phòng mổ", "Hỗ trợ"]},
  {"name": "Dental Assistant and Therapist", "keywords": ["Trợ lý nha khoa", "Điều trị nha khoa", "Răng", "Y tế", "Phụ tá"]},
  {"name": "Sports Professional", "keywords": ["Vận động viên chuyên nghiệp", "Thể thao", "Thi đấu", "Chuyên nghiệp", "Thể chất"]},
  {"name": "Physical Education Teacher", "keywords": ["Giáo viên thể dục", "Thể thao", "Giáo dục", "Thể chất", "Dạy học"]},
  {"name": "Telecommunications Technician", "keywords": ["Kỹ thuật viên viễn thông", "Viễn thông", "Mạng", "Truyền thông", "Lắp đặt"]},
  {"name": "Fire-Fighter", "keywords": ["Lính cứu hỏa", "Cứu hỏa", "Khẩn cấp", "Cứu hộ", "An toàn"]},
  {"name": "Crop Grower", "keywords": ["Người trồng trọt", "Cây trồng", "Nông nghiệp", "Canh tác", "Sản xuất"]},
  {"name": "Animal Husbandry Specialist", "keywords": ["Chuyên gia chăn nuôi", "Vật nuôi", "Gia súc", "Nông nghiệp", "Chăn nuôi"]},
  {"name": "Apiarist and Sericulturist", "keywords": ["Nuôi ong và tằm", "Ong", "Tằm", "Nông nghiệp", "Sản xuất"]},
  {"name": "Plumber and Pipe Fitter", "keywords": ["Thợ ống nước", "Hệ thống nước", "Lắp đặt", "Sửa chữa", "Xây dựng"]},
  {"name": "Tool and Die Maker", "keywords": ["Thợ làm khuôn", "Công cụ", "Chế tạo", "Cơ khí", "Khuôn mẫu"]},
  {"name": "Carpenter and Joiner", "keywords": ["Thợ mộc", "Đồ gỗ", "Xây dựng", "Mộc", "Nội thất"]},
  {"name": "HVAC Technician", "keywords": ["Kỹ thuật viên điều hòa", "Sưởi", "Làm mát", "Thông gió", "Lắp đặt"]},
  {"name": "Welder and Flame Cutter", "keywords": ["Thợ hàn", "Hàn", "Cắt kim loại", "Chế tạo", "Xây dựng"]},
  {"name": "Mechanical Engineering Technician (Ship Building)", "keywords": ["Kỹ thuật viên đóng tàu", "Đóng tàu", "Tàu biển", "Hàng hải", "Cơ khí"]},
  {"name": "Mechanical Technician (Metal Working)", "keywords": ["Kỹ thuật viên kim loại", "Gia công", "Kim loại", "Cơ khí", "Sản xuất"]},
  {"name": "Auto Mechanic", "keywords": ["Thợ sửa ô tô", "Cơ khí", "Ô tô", "Sửa chữa", "Bảo dưỡng"]},
  {"name": "Mechanical Engineering Technician: Agricultural", "keywords": ["Kỹ thuật viên máy nông nghiệp", "Máy nông nghiệp", "Cơ khí", "Canh tác", "Thiết bị"]},
  {"name": "Underwater Diver", "keywords": ["Thợ lặn", "Lặn", "Dưới nước", "Thám hiểm", "Hàng hải"]},
  {"name": "Locomotive Driver", "keywords": ["Lái tàu hỏa", "Đường sắt", "Tàu hỏa", "Vận tải", "Điều khiển"]},
  {"name": "Car, Taxi and Light Truck Driver", "keywords": ["Tài xế ô tô taxi", "Lái xe", "Vận tải", "Dịch vụ", "Giao thông"]},
  {"name": "Bus and Tram Driver", "keywords": ["Tài xế xe buýt", "Xe tram", "Giao thông công cộng", "Vận tải", "Hành khách"]},
  {"name": "Army officer", "keywords": ["Sĩ quan quân đội", "Quân đội", "Quân sự", "Lãnh đạo", "Quốc phòng"]},
  {"name": "Forestry Worker", "keywords": ["Công nhân lâm nghiệp", "Rừng", "Khai thác", "Bảo vệ", "Làm việc ngoài trời"]},
  {"name": "Aquaculture and Fishery Worker", "keywords": ["Người nuôi trồng thủy sản", "Đánh bắt", "Thủy sản", "Cá", "Nuôi trồng"]},
  {"name": "Mason", "keywords": ["Thợ nề", "Xây dựng", "Gạch đá", "Công trình", "Xây"]},
  {"name": "Building Finisher", "keywords": ["Thợ hoàn thiện công trình", "Hoàn thiện", "Xây dựng", "Nội thất", "Trang trí"]},
  {"name": "Painter", "keywords": ["Thợ sơn", "Sơn", "Xây dựng", "Trang trí", "Hoàn thiện"]},
  {"name": "Electrical Installation and Repair Worker", "keywords": ["Thợ điện", "Lắp đặt", "Sửa chữa", "Hệ thống điện", "Bảo trì"]},
  {"name": "Electronics and Telecommunications Installation and Repair Worker", "keywords": ["Thợ điện tử viễn thông", "Lắp đặt", "Sửa chữa", "Điện tử", "Viễn thông"]},
  {"name": "Assembler", "keywords": ["Thợ lắp ráp", "Lắp ráp", "Sản xuất", "Nhà máy", "Linh kiện"]}
]

# Hàm chuẩn hóa tiêu đề sử dụng từ khóa
def normalize_title(title):
    if title is None:
        return "Unknown"
    
    # Chuẩn hóa title
    normalized = re.sub(r'\([^)]*\)', '', title)  # Xóa nội dung trong ngoặc ()
    normalized = re.sub(r'-.*$', '', normalized)   # Xóa nội dung sau dấu gạch ngang
    normalized = normalized.strip().lower()
    
    # Nếu title trống, trả về Unknown
    if not normalized:
        return "Unknown"
    
    best_match = None
    best_match_score = 0
    
    # Tìm kiếm theo số lượng từ khóa khớp
    for profession in professions:
        # Đếm số từ khóa trong title
        matched_keywords = sum(1 for keyword in profession["keywords"] if keyword.lower() in normalized)
        
        # Tính tỷ lệ khớp
        if len(profession["keywords"]) > 0:
            match_ratio = matched_keywords / len(profession["keywords"])
            
            # Cân nhắc tỷ lệ khớp và số lượng từ khóa khớp
            score = match_ratio * matched_keywords
            
            # Lấy tiêu đề khớp nhất (cần có ít nhất 2 từ khóa khớp hoặc nếu chỉ có 1 từ khóa thì phải khớp hoàn toàn)
            if score > best_match_score and matched_keywords >= min(2, len(profession["keywords"])):
                best_match_score = score
                best_match = profession["name"]
    
    return best_match if best_match else "Unknown"

# Đăng ký UDF
normalize_title_udf = udf(normalize_title, StringType())

# Áp dụng UDF để chuẩn hóa tiêu đề
job_data_normalized = job_data.withColumn('job_title_en', normalize_title_udf(job_data['job_title']))

# Sử dụng pandas để tránh lỗi Hadoop
pandas_df = job_data_normalized.toPandas()
pandas_df.to_csv('jobs_data_careerviet2_final_normalized.csv', index=False)

print("Đã hoàn thành việc chuẩn hóa tiêu đề và lưu vào file 'jobs_data_careerviet2_final_normalized.csv'.")

# Dừng SparkSession
spark.stop()

In [ ]:
import re
from fuzzywuzzy import fuzz
professions=[
{"name": "University Teacher", "keywords": ["Giáo", "sư", "Đại", "học", "Giảng", "dạy", "Học", "thuật", "dục", "Chuyên", "viên"]},
    {"name": "College Teacher", "keywords": ["Giảng", "viên", "Cao", "đẳng", "Giáo", "dục", "Đào", "tạo", "dạy"]},
    {"name": "Secondary Vocational and Professional School Teacher", "keywords": ["Dạy", "nghề", "Chuyên", "nghiệp", "Giáo", "viên", "Đào", "tạo", "Kỹ", "năng"]},
    {"name": "Upper Secondary Education Teacher", "keywords": ["Trung", "học", "phổ", "thông", "Giáo", "viên", "dục", "Chương", "trình"]},
    {"name": "Lower Secondary Education Teacher", "keywords": ["Trung", "học", "cơ", "sở", "Giáo", "viên", "dục", "Thanh", "thiếu", "niên", "Dạy"]},
    {"name": "Foreign Language Teacher", "keywords": ["Ngoại", "ngữ", "Giáo", "viên", "Ngôn", "Quốc", "tế", "Giao", "tiếp"]},
    {"name": "Public Relations Officer", "keywords": ["Quan", "hệ", "công", "chúng", "Truyền", "thông", "PR", "Doanh", "nghiệp", "Hình", "ảnh"]},
    {"name": "Lawyer", "keywords": ["Luật", "sư", "Pháp", "luật", "Tư", "vấn", "Biện", "hộ", "pháp"]},
    {"name": "Archivist and Curator", "keywords": ["Lưu", "trữ", "Quản", "lý", "bảo", "tàng", "Bảo", "quản", "Hiện", "vật", "Lịch", "sử"]},
    {"name": "Librarian and Library Clerk", "keywords": ["Thủ", "thư", "Thư", "viện", "Sách", "Tài", "liệu", "Quản", "lý", "thông", "tin"]},
    {"name": "Editor/Publisher", "keywords": ["Biên", "tập", "viên", "Xuất", "bản", "Báo", "chí", "Nội", "dung", "Sách"]},
    {"name": "Social Scientist", "keywords": ["Nhà", "khoa", "học", "xã", "hội", "Nghiên", "cứu", "Xã", "Phân", "tích", "Hành", "vi"]},
    {"name": "Archaeologist", "keywords": ["Nhà", "khảo", "cổ", "học", "Khai", "quật", "Di", "tích", "Lịch", "sử", "Cổ", "vật"]},
    {"name": "Historian", "keywords": ["Sử", "học", "Nghiên", "cứu", "lịch", "sử", "Tài", "liệu", "Phân", "tích", "Quá", "khứ"]},
    {"name": "Script Writer", "keywords": ["Biên", "kịch", "Kịch", "bản", "Sáng", "tạo", "Phim", "ảnh", "Truyền", "thông"]},
    {"name": "Journalist", "keywords": ["Nhà", "báo", "Tin", "tức", "Phóng", "viên", "Truyền", "thông", "Thời", "sự"]},
    {"name": "Language Specialist", "keywords": ["Chuyên", "gia", "ngôn", "ngữ", "Ngôn", "học", "Giao", "tiếp", "Phiên", "dịch", "Phân", "tích"]},
    {"name": "Translator and Interpreter", "keywords": ["Biên", "dịch", "Phiên", "Ngôn", "ngữ", "Giao", "tiếp", "Quốc", "tế"]},
    {"name": "Director: Film/Stage", "keywords": ["Đạo", "diễn", "Phim", "Sân", "khấu", "Sản", "xuất", "Giải", "trí"]},
    {"name": "Announcer: Radio, Television and Other media", "keywords": ["Người", "dẫn", "chương", "trình", "Phát", "thanh", "Truyền", "hình", "thông", "MC"]},
    {"name": "Health Information Administrator", "keywords": ["Quản", "lý", "thông", "tin", "y", "tế", "Y", "Hồ", "sơ", "Dữ", "liệu"]},
    {"name": "Museologist", "keywords": ["Chuyên", "gia", "bảo", "tàng", "Triển", "lãm", "Bảo", "Di", "sản", "Văn", "hóa"]},
    {"name": "Secretary", "keywords": ["Thư", "ký", "Hành", "chính", "Văn", "phòng", "Trợ", "lý", "Điều", "hành"]},
    {"name": "Broadcasting and Audiovisual Technicians", "keywords": ["Kỹ", "thuật", "viên", "phát", "thanh", "nghe", "nhìn", "Truyền", "hình", "Thiết", "bị"]},
    {"name": "Call Centre Operator", "keywords": ["Tổng", "đài", "viên", "Chăm", "sóc", "khách", "hàng", "Điện", "thoại", "Hỗ", "trợ", "Liên", "lạc"]},
    {"name": "Tour Guide", "keywords": ["Hướng", "dẫn", "viên", "du", "lịch", "Du", "Lữ", "hành", "Văn", "hóa", "Thông", "tin"]},
    {"name": "Marketing and Sales Assistant", "keywords": ["Trợ", "lý", "tiếp", "thị", "bán", "hàng", "Marketing", "Kinh", "doanh", "Khách"]},
    {"name": "Astronomer", "keywords": ["Nhà", "thiên", "văn", "học", "Vũ", "trụ", "Thiên", "Nghiên", "cứu", "Khoa"]},
    {"name": "Meteorologist", "keywords": ["Nhà", "khí", "tượng", "học", "Thời", "tiết", "Khí", "hậu", "Dự", "báo", "Khoa"]},
    {"name": "Chemist", "keywords": ["Nhà", "hóa", "học", "Hóa", "chất", "Phòng", "thí", "nghiệm", "Nghiên", "cứu", "Phân", "tích"]},
    {"name": "Geologist", "keywords": ["Nhà", "địa", "chất", "học", "Địa", "Khoáng", "sản", "Đất", "đá", "Khoa"]},
    {"name": "Geographer", "keywords": ["Nhà", "địa", "lý", "học", "Bản", "đồ", "Không", "gian", "Môi", "trường", "Địa", "hình"]},
    {"name": "Mathematician", "keywords": ["Nhà", "toán", "học", "Toán", "Tính", "Lý", "thuyết", "Logic"]},
    {"name": "Statistician", "keywords": ["Nhà", "thống", "kê", "Dữ", "liệu", "Phân", "tích", "Xác", "suất", "Nghiên", "cứu"]},
    {"name": "Actuarial Scientist", "keywords": ["Chuyên", "gia", "tính", "toán", "bảo", "hiểm", "Phân", "tích", "rủi", "ro", "Bảo", "Thống", "kê", "Tài", "chính"]},
    {"name": "Biologist", "keywords": ["Nhà", "sinh", "học", "Sinh", "vật", "Nghiên", "cứu", "Khoa", "Phòng", "thí", "nghiệm"]},
    {"name": "Marine Scientist", "keywords": ["Nhà", "khoa", "học", "biển", "Đại", "dương", "Biển", "Sinh", "vật", "Nghiên", "cứu"]},
    {"name": "Engineer: Biotechnologist", "keywords": ["Kỹ", "sư", "công", "nghệ", "sinh", "học", "Sinh", "Công", "Nghiên", "cứu", "thuật"]},
    {"name": "Biomedical Engineer", "keywords": ["Kỹ", "sư", "y", "sinh", "Y", "tế", "Thiết", "bị", "Công", "nghệ", "học"]},
    {"name": "Agricultural Scientist", "keywords": ["Nhà", "khoa", "học", "nông", "nghiệp", "Canh", "tác", "Nông", "Cây", "trồng", "Nghiên", "cứu"]},
    {"name": "Engineer: Marine/Fishery Scientists and Aquaculturists", "keywords": ["Kỹ", "sư", "thủy", "sản", "Ngư", "nghiệp", "Nuôi", "trồng", "Biển", "Môi", "trường"]},
    {"name": "Forestry Advisor", "keywords": ["Cố", "vấn", "lâm", "nghiệp", "Rừng", "Bảo", "tồn", "Môi", "trường", "Quản", "lý"]},
    {"name": "Environmental Specialist", "keywords": ["Chuyên", "gia", "môi", "trường", "Bảo", "vệ", "Sinh", "thái", "Bền", "vững", "Đánh", "giá"]},
    {"name": "Industrial Engineer", "keywords": ["Kỹ", "sư", "công", "nghiệp", "Tối", "ưu", "hóa", "Hệ", "thống", "Hiệu", "quả", "Sản", "xuất"]},
    {"name": "Food and Drink Technologist", "keywords": ["Kỹ", "thuật", "viên", "thực", "phẩm", "đồ", "uống", "Chế", "biến", "Công", "nghệ", "Chất", "lượng", "Sản", "xuất"]},
    {"name": "Civil Engineer", "keywords": ["Kỹ", "sư", "xây", "dựng", "Công", "trình", "Cơ", "sở", "hạ", "tầng", "Thiết", "kế", "Xây"]},
    {"name": "Automobile Engineer", "keywords": ["Kỹ", "sư", "ô", "tô", "Xe", "hơi", "Thiết", "kế", "Cơ", "khí", "Phương", "tiện"]},
    {"name": "Mechanical Engineer", "keywords": ["Kỹ", "sư", "cơ", "khí", "Máy", "móc", "Thiết", "kế", "Hệ", "thống", "Cơ"]},
    {"name": "Chemical Engineer", "keywords": ["Kỹ", "sư", "hóa", "học", "Hóa", "chất", "Quy", "trình", "Công", "nghiệp", "Vật", "liệu"]},
    {"name": "Metallurgical Engineer", "keywords": ["Kỹ", "sư", "luyện", "kim", "Kim", "loại", "Vật", "liệu", "Chế", "biến", "Công", "nghiệp"]},
    {"name": "Materials Engineer", "keywords": ["Kỹ", "sư", "vật", "liệu", "Chất", "Phát", "triển", "Nghiên", "cứu", "Tính", "chất"]},
    {"name": "Electrical Engineer", "keywords": ["Kỹ", "sư", "điện", "Điện", "lực", "Hệ", "thống", "Thiết", "kế", "Năng", "lượng"]},
    {"name": "Electronics Engineer", "keywords": ["Kỹ", "sư", "điện", "tử", "Mạch", "Hệ", "thống", "Thiết", "kế", "Điện"]},
    {"name": "Engineer: Telecommunication", "keywords": ["Kỹ", "sư", "viễn", "thông", "Mạng", "lưới", "Truyền", "Hệ", "thống", "Kết", "nối"]},
    {"name": "Aerospace/Aeronautical Engineer", "keywords": ["Kỹ", "sư", "hàng", "không", "vũ", "trụ", "Máy", "bay", "thuật", "Thiết", "kế", "Bay"]},
    {"name": "Veterinarian", "keywords": ["Bác", "sĩ", "thú", "y", "Động", "vật", "Y", "tế", "Điều", "trị", "Chăm", "sóc"]},
    {"name": "Veterinarian technician", "keywords": ["Kỹ", "thuật", "viên", "thú", "y", "Động", "vật", "Hỗ", "trợ", "Chăm", "sóc", "Y", "tế"]},
    {"name": "Pharmacist", "keywords": ["Dược", "sĩ", "Thuốc", "Y", "tế", "phẩm", "Điều", "chế"]},
    {"name": "Accountant", "keywords": ["Kế", "Toán", "Viên", "Tài", "Chính", "Sổ", "Sách", "Kiểm"]},
    {"name": "Financial Analyst", "keywords": ["Chuyên", "viên", "phân", "tích", "tài", "chính", "Đầu", "tư", "Thị", "trường", "Tài", "Kinh", "doanh"]},
    {"name": "Administration Specialist: Oil and Gas", "keywords": ["Chuyên", "viên", "quản", "lý", "dầu", "khí", "Dầu", "mỏ", "Khí", "đốt", "Quản", "Năng", "lượng"]},
    {"name": "Administration Specialist: Transport", "keywords": ["Chuyên", "viên", "quản", "lý", "vận", "tải", "Vận", "chuyển", "Logistics", "Quản", "hành"]},
    {"name": "Administration Specialist: Energy Management", "keywords": ["Chuyên", "viên", "quản", "lý", "năng", "lượng", "Năng", "Hiệu", "quả", "Quản", "Tài", "nguyên"]},
    {"name": "Computer Scientist: Software Developer", "keywords": ["Lập", "trình", "viên", "Phần", "mềm", "Công", "nghệ", "IT"]},
    {"name": "Economist", "keywords": ["Nhà", "kinh", "tế", "học", "Kinh", "Thị", "trường", "Phân", "tích", "Chính", "sách"]},
    {"name": "International Trade (Exporter – Importer)", "keywords": ["Thương", "mại", "quốc", "tế", "Xuất", "khẩu", "Nhập", "Kinh", "doanh", "Quốc"]},
    {"name": "Investment Banker", "keywords": ["Chuyên", "viên", "ngân", "hàng", "đầu", "tư", "Đầu", "Tài", "chính", "Thị", "trường", "Sáp", "nhập"]},
    {"name": "Sound Engineer", "keywords": ["Kỹ", "sư", "âm", "thanh", "Âm", "nhạc", "Ghi", "Sản", "xuất", "Thiết", "bị"]},
    {"name": "Bank Teller and Related Clerks", "keywords": ["Giao", "dịch", "viên", "ngân", "hàng", "Tài", "chính", "Khách", "Ngân"]},
    {"name": "Information Technology Teacher", "keywords": ["Giáo", "viên", "công", "nghệ", "thông", "tin", "IT", "Giảng", "dạy", "Máy", "tính", "dục"]},
    {"name": "Graphic and Multi Media Designer", "keywords": ["Nhà", "thiết", "kế", "đồ", "họa", "Thiết", "Đa", "phương", "tiện", "Hình", "ảnh", "Sáng", "tạo"]},
    {"name": "Computer database and network administrators", "keywords": ["Quản", "trị", "mạng", "Cơ", "sở", "dữ", "liệu", "Hệ", "thống", "IT", "lý"]},
    {"name": "ICT technician", "keywords": ["Kỹ", "thuật", "viên", "CNTT", "Hỗ", "trợ", "Máy", "tính", "Công", "nghệ"]},
    {"name": "Treasurer", "keywords": ["Thủ", "quỹ", "Tài", "chính", "Quản", "lý", "Ngân", "sách", "Quỹ"]},
    {"name": "Packaging Technologists", "keywords": ["Chuyên", "gia", "đóng", "gói", "Bao", "bì", "Thiết", "kế", "Sản", "phẩm", "Công", "nghệ"]},
    {"name": "Printing Technologists", "keywords": ["Chuyên", "gia", "in", "ấn", "In", "Sản", "xuất", "Công", "nghệ", "Ấn", "phẩm"]},
    {"name": "Paper Technologists", "keywords": ["Chuyên", "gia", "công", "nghệ", "giấy", "Giấy", "Sản", "xuất", "Vật", "liệu", "Công"]},
    {"name": "Garment and Related Trades Technicians", "keywords": ["Kỹ", "thuật", "viên", "may", "mặc", "Dệt", "Thời", "trang", "Sản", "xuất"]},
    {"name": "Leather and Footwear Technologists", "keywords": ["Chuyên", "gia", "da", "giày", "Da", "Giày", "dép", "Sản", "xuất", "Công", "nghệ"]},
    {"name": "Cosmetologists (Beauticians)", "keywords": ["Chuyên", "viên", "làm", "đẹp", "Thẩm", "mỹ", "Chăm", "sóc", "da", "Làm", "Mỹ", "phẩm"]},
    {"name": "Architects (Building)", "keywords": ["Kiến", "trúc", "sư", "Thiết", "kế", "Xây", "dựng", "Công", "trình", "Quy", "hoạch"]},
    {"name": "Landscape Designers/Architects", "keywords": ["Kiến", "trúc", "sư", "cảnh", "quan", "Thiết", "kế", "Môi", "trường", "Ngoại", "thất", "Cảnh"]},
    {"name": "Product/Industrial Designers", "keywords": ["Nhà", "thiết", "kế", "sản", "phẩm", "Thiết", "công", "nghiệp", "Sản", "Sáng", "tạo", "Phát", "triển"]},
    {"name": "Fashion Designers", "keywords": ["Nhà", "thiết", "kế", "thời", "trang", "Thời", "Trang", "phục", "Thiết", "Xu", "hướng"]},
    {"name": "Cartographers and Surveyors", "keywords": ["Nhà", "đo", "đạc", "bản", "đồ", "Khảo", "sát", "Bản", "Địa", "lý", "Đo", "lường"]},
    {"name": "Draughts persons", "keywords": ["Họa", "viên", "kỹ", "thuật", "Vẽ", "Bản", "vẽ", "Kỹ", "Thiết", "kế"]},
    {"name": "Photographers", "keywords": ["Nhiếp", "ảnh", "gia", "Chụp", "Hình", "Máy", "Sáng", "tạo"]},
    {"name": "Interior Designers and Decorators", "keywords": ["Nhà", "thiết", "kế", "nội", "thất", "Trang", "trí", "Không", "gian", "Nội", "Thẩm", "mỹ"]},
    {"name": "Crafts Designers", "keywords": ["Nhà", "thiết", "kế", "thủ", "công", "Thủ", "Nghệ", "thuật", "Sáng", "tạo", "Mỹ", "nghệ"]},
    {"name": "Fine Artists", "keywords": ["Họa", "sĩ", "Nghệ", "thuật", "Hội", "họa", "Điêu", "khắc", "Sáng", "tạo"]},
    {"name": "Commercial Artists", "keywords": ["Họa", "sĩ", "thương", "mại", "Quảng", "cáo", "Mỹ", "thuật", "Thiết", "kế", "Thương"]},
    {"name": "Cinematographers", "keywords": ["Quay", "phim", "Điện", "ảnh", "Máy", "quay", "Hình", "Sản", "xuất"]},
    {"name": "Gardeners, Horticultural and Nursery Growers", "keywords": ["Nhà", "làm", "vườn", "Làm", "Cây", "trồng", "Trồng", "trọt", "cảnh"]},
    {"name": "Jewelry Designers", "keywords": ["Nhà", "thiết", "kế", "trang", "sức", "Trang", "Mỹ", "nghệ", "Thiết", "Thủ", "công"]},
    {"name": "Potters and Related Workers", "keywords": ["Thợ", "gốm", "Gốm", "sứ", "Thủ", "công", "Nghệ", "thuật", "Sản", "xuất"]},
    {"name": "Sign writers, Decorative Painters and Related Workers", "keywords": ["Thợ", "vẽ", "biển", "hiệu", "Vẽ", "trang", "trí", "Mỹ", "thuật", "Biển", "Nghệ"]},
    {"name": "Handicraft Workers in Furniture and Home Furnishings Industries", "keywords": ["Thợ", "thủ", "công", "nội", "thất", "Đồ", "gỗ", "Nội", "Thủ", "Sản", "xuất"]},
    {"name": "Handicraft Workers in Textile-Garment and Leather-Footwear Industries", "keywords": ["Thợ", "thủ", "công", "dệt", "may", "Da", "giày", "Thủ", "Dệt", "Sản", "xuất"]},
    {"name": "Machine Operators in Textile-Garment and Leather-Footwear Industries", "keywords": ["Công", "nhân", "vận", "hành", "máy", "dệt", "may", "Da", "giày", "Sản", "xuất", "Máy", "móc", "Nhà"]},
    {"name": "Allopathic Medical Doctor", "keywords": ["Bác", "sĩ", "Y", "khoa", "tế", "Khám", "chữa", "bệnh", "Điều", "trị"]},
    {"name": "Nurse and Midwife", "keywords": ["Y", "tá", "Hộ", "sinh", "Điều", "dưỡng", "tế", "Chăm", "sóc"]},
    {"name": "Medical Doctor: Traditional Medicine", "keywords": ["Bác", "sĩ", "y", "học", "cổ", "truyền", "Đông", "Y", "Chữa", "bệnh", "Truyền", "thống"]},
    {"name": "Dentist", "keywords": ["Nha", "sĩ", "Răng", "khoa", "Y", "tế", "Điều", "trị"]},
    {"name": "Occupational Therapist", "keywords": ["Nhà", "trị", "liệu", "hoạt", "động", "Phục", "hồi", "Chức", "năng", "Y", "tế", "Điều"]},
    {"name": "Physiotherapist", "keywords": ["Nhà", "vật", "lý", "trị", "liệu", "Phục", "hồi", "Chức", "năng", "Điều"]},
    {"name": "Dietician and Nutritionist", "keywords": ["Chuyên", "gia", "dinh", "dưỡng", "Dinh", "Thực", "phẩm", "Sức", "khỏe", "Chế", "độ", "ăn"]},
    {"name": "Audiologist and Speech Therapist", "keywords": ["Chuyên", "gia", "thính", "học", "Trị", "liệu", "ngôn", "ngữ", "Nghe", "Nói", "Y", "tế"]},
    {"name": "Pre-School Teacher", "keywords": ["Giáo", "viên", "mầm", "non", "Trẻ", "em", "dục", "Mầm", "Dạy", "học"]},
    {"name": "Primary School Teacher", "keywords": ["Giáo", "viên", "tiểu", "học", "Tiểu", "dục", "Trẻ", "em", "Dạy"]},
    {"name": "Special Needs Teacher", "keywords": ["Giáo", "viên", "giáo", "dục", "đặc", "biệt", "Đặc", "Khuyết", "tật", "Hỗ", "trợ"]},
    {"name": "Administration Specialist: Small & Medium Business Management", "keywords": ["Chuyên", "viên", "quản", "lý", "doanh", "nghiệp", "vừa", "và", "nhỏ", "SME", "Quản", "Kinh", "Vận", "hành"]},
    {"name": "Agribusiness Management", "keywords": ["Quản", "lý", "nông", "nghiệp", "Nông", "Kinh", "doanh", "Thương", "mại"]},
    {"name": "Sales and Marketing Manager", "keywords": ["Giám", "đốc", "kinh", "doanh", "tiếp", "thị", "Marketing", "Bán", "hàng", "Chiến", "lược", "Quản", "lý"]},
    {"name": "Retail and Wholesale Manager", "keywords": ["Quản", "lý", "bán", "lẻ", "sỉ", "Bán", "Thương", "mại"]},
    {"name": "Hotel/HORECA Manager", "keywords": ["Quản", "lý", "khách", "sạn", "nhà", "hàng", "Khách", "Dịch", "vụ", "Du", "lịch"]},
    {"name": "Agriculture Manager", "keywords": ["Quản", "lý", "nông", "nghiệp", "Nông", "Canh", "tác", "Sản", "xuất"]},
    {"name": "Hospital Administrator", "keywords": ["Quản", "lý", "bệnh", "viện", "Y", "tế", "Bệnh", "Điều", "hành"]},
    {"name": "Human Resource Manager", "keywords": ["Giám", "đốc", "nhân", "sự", "Nhân", "Tuyển", "dụng", "Quản", "lý", "viên"]},
    {"name": "Administration Specialist: Urban Management", "keywords": ["Chuyên", "viên", "quản", "lý", "đô", "thị", "Đô", "Quy", "hoạch", "Quản", "Thành", "phố"]},
    {"name": "Labour Relations Officer", "keywords": ["Chuyên", "viên", "quan", "hệ", "lao", "động", "Lao", "Nhân", "sự", "Công", "đoàn", "Quan"]},
    {"name": "Legal Services Specialist", "keywords": ["Chuyên", "viên", "dịch", "vụ", "pháp", "lý", "Pháp", "luật", "Tư", "vấn", "Dịch", "Luật"]},
    {"name": "Psychologist", "keywords": ["Nhà", "tâm", "lý", "học", "Tâm", "Sức", "khỏe", "thần", "Trị", "liệu", "Đánh", "giá"]},
    {"name": "Counsellor", "keywords": ["Nhà", "tư", "vấn", "Tư", "Hỗ", "trợ", "Tâm", "lý", "Trị", "liệu"]},
    {"name": "Career Counsellor", "keywords": ["Cố", "vấn", "nghề", "nghiệp", "Hướng", "Tư", "Nghề", "Việc", "làm"]},
    {"name": "Medical Technologist: Medical Imaging", "keywords": ["Kỹ", "thuật", "viên", "chẩn", "đoán", "hình", "ảnh", "Y", "tế", "Chẩn", "Hình", "X-quang"]},
    {"name": "Medical Technologist: Optometry", "keywords": ["Kỹ", "thuật", "viên", "nhãn", "khoa", "Mắt", "Thị", "lực", "Y", "tế", "Nhãn"]},
    {"name": "Medical Technologist", "keywords": ["Kỹ", "thuật", "viên", "y", "tế", "Y", "Xét", "nghiệm", "Chẩn", "đoán"]},
    {"name": "Insurance Agent", "keywords": ["Đại", "lý", "bảo", "hiểm", "Bảo", "Tư", "vấn", "Hợp", "đồng", "Dịch", "vụ"]},
    {"name": "Event Manager", "keywords": ["Quản", "lý", "sự", "kiện", "Sự", "Tổ", "chức", "Điều", "phối", "Kế", "hoạch"]},
    {"name": "Administration Specialist: Real Estate", "keywords": ["Chuyên", "viên", "quản", "lý", "bất", "động", "sản", "Bất", "Địa", "ốc", "Quản", "Tài"]},
    {"name": "Social Worker", "keywords": ["Nhân", "viên", "xã", "hội", "Xã", "Hỗ", "trợ", "Cộng", "đồng", "Phúc", "lợi"]},
    {"name": "Travel and Tourism Agent", "keywords": ["Đại", "lý", "du", "lịch", "Du", "Lữ", "hành", "Đặt", "tour", "Dịch", "vụ"]},
    {"name": "Air Hostess/Steward", "keywords": ["Tiếp", "viên", "hàng", "không", "Máy", "bay", "Phục", "vụ", "Hàng", "Dịch"]},
    {"name": "Food Processing Workers", "keywords": ["Công", "nhân", "chế", "biến", "thực", "phẩm", "Thực", "Chế", "Sản", "xuất", "Nhà", "máy"]},
    {"name": "Cook", "keywords": ["Đầu", "bếp", "Nấu", "ăn", "Ẩm", "thực", "Nhà", "hàng", "Chế", "biến"]},
    {"name": "Cook Assistant", "keywords": ["Phụ", "bếp", "Hỗ", "trợ", "Nhà", "Chế", "biến", "hàng"]},
    {"name": "Waiter", "keywords": ["Bồi", "bàn", "Phục", "vụ", "Nhà", "hàng", "Dịch", "Khách"]},
    {"name": "Receptionist", "keywords": ["Lễ", "tân", "Tiếp", "Đón", "tiếp", "Văn", "phòng", "Khách", "hàng"]},
    {"name": "Office clerk", "keywords": ["Nhân", "viên", "văn", "phòng", "Hành", "chính", "Văn", "thư", "Giấy", "tờ"]},
    {"name": "Aircraft Maintenance Engineer", "keywords": ["Kỹ", "sư", "bảo", "trì", "máy", "bay", "Hàng", "không", "Bảo", "dưỡng", "thuật", "Máy"]},
    {"name": "Engineer Mining", "keywords": ["Kỹ", "sư", "khai", "thác", "mỏ", "Khai", "Mỏ", "Địa", "chất", "Tài", "nguyên"]},
    {"name": "Oil and Gas Engineer", "keywords": ["Kỹ", "sư", "dầu", "khí", "Dầu", "mỏ", "Khí", "đốt", "Năng", "lượng", "Khai", "thác"]},
    {"name": "Orthotist and Prosthetist", "keywords": ["Chuyên", "gia", "chỉnh", "hình", "Bộ", "phận", "giả", "Y", "tế", "Phục", "hồi", "Thiết", "bị"]},
    {"name": "Mechatronics Engineer", "keywords": ["Kỹ", "sư", "cơ", "điện", "tử", "Tự", "động", "hóa", "Robot", "Hệ", "thống", "Điều", "khiển"]},
    {"name": "Merchant Navy Engineer", "keywords": ["Kỹ", "sư", "hàng", "hải", "Tàu", "biển", "Hàng", "Vận", "hành", "thuật"]},
    {"name": "Police Officer", "keywords": ["Cảnh", "sát", "An", "ninh", "Pháp", "luật", "Trật", "tự", "Hình", "sự"]},
    {"name": "Dancer and Choreographer", "keywords": ["Vũ", "công", "Biên", "đạo", "múa", "Nghệ", "thuật", "Nhảy", "Biểu", "diễn"]},
    {"name": "Civil Engineering Technician", "keywords": ["Kỹ", "thuật", "viên", "xây", "dựng", "Công", "trình", "Xây", "Hạ", "tầng"]},
    {"name": "Irrigation technician", "keywords": ["Kỹ", "thuật", "viên", "thủy", "lợi", "Tưới", "tiêu", "Nông", "nghiệp", "Thủy", "Hệ", "thống"]},
    {"name": "Electrical Engineering Technician (Information-Communication Technology)", "keywords": ["Kỹ", "thuật", "viên", "điện", "ICT", "Công", "nghệ", "thông", "tin", "Điện", "tử", "Truyền"]},
    {"name": "Electrical Engineering Technician and Operator (Power Generation and Transmission)", "keywords": ["Kỹ", "thuật", "viên", "điện", "lực", "Phát", "Truyền", "tải", "Điện", "Vận", "hành"]},
    {"name": "Electrical Engineering Technician (Turbines)", "keywords": ["Kỹ", "thuật", "viên", "tuabin", "điện", "Tuabin", "Điện", "Năng", "lượng"]},
    {"name": "Computer Hardware Technologist", "keywords": ["Chuyên", "gia", "phần", "cứng", "máy", "tính", "Phần", "Máy", "Thiết", "bị", "Công", "nghệ"]},
    {"name": "Ship Radio Officer", "keywords": ["Sĩ", "quan", "vô", "tuyến", "tàu", "biển", "Thông", "tin", "Tàu", "Vô", "Hàng", "hải"]},
    {"name": "Mechanical Engineering Technician (Manufacture, Assembly and Repair)", "keywords": ["Kỹ", "thuật", "viên", "cơ", "khí", "Sản", "xuất", "Lắp", "ráp", "Sửa", "chữa", "Máy", "móc"]},
    {"name": "Transport Technician", "keywords": ["Kỹ", "thuật", "viên", "vận", "tải", "Vận", "Phương", "tiện", "Bảo", "trì"]},
    {"name": "Mining Technician", "keywords": ["Kỹ", "thuật", "viên", "khai", "thác", "mỏ", "Khai", "Mỏ", "Hoạt", "động", "Tài", "nguyên"]},
    {"name": "Metallurgical Technician", "keywords": ["Kỹ", "thuật", "viên", "luyện", "kim", "Kim", "loại", "Vật", "liệu", "Xử", "lý"]},
    {"name": "Mechanical Engineering Technician: Operator", "keywords": ["Kỹ", "thuật", "viên", "vận", "hành", "cơ", "khí", "Vận", "Máy", "móc", "Cơ", "Thiết", "bị"]},
    {"name": "Petroleum and Natural Gas Refining Plant Operator", "keywords": ["Người", "vận", "hành", "nhà", "máy", "lọc", "dầu", "khí", "Dầu", "mỏ", "Khí", "đốt", "Lọc", "Vận"]},
    {"name": "Environmental Protection Technician", "keywords": ["Kỹ", "thuật", "viên", "bảo", "vệ", "môi", "trường", "Môi", "Bảo", "Giám", "sát", "Sinh", "thái"]},
    {"name": "Forester", "keywords": ["Kiểm", "lâm", "Rừng", "Bảo", "tồn", "Quản", "lý", "Môi", "trường"]},
    {"name": "Merchant Navy: Mariner", "keywords": ["Thủy", "thủ", "hàng", "hải", "Tàu", "biển", "Hàng", "Đi", "Vận", "chuyển"]},
    {"name": "Pilot", "keywords": ["Phi", "công", "Lái", "máy", "bay", "Hàng", "không", "Bay", "Vận", "tải"]},
    {"name": "Air Traffic Controller", "keywords": ["Kiểm", "soát", "viên", "không", "lưu", "Không", "Hàng", "An", "toàn", "Điều", "hành"]},
    {"name": "Medical Technologist: Nuclear Medicine Technology", "keywords": ["Kỹ", "thuật", "viên", "y", "học", "hạt", "nhân", "Hạt", "Y", "tế", "Chẩn", "đoán", "Xạ"]},
    {"name": "Laboratory Technician", "keywords": ["Kỹ", "thuật", "viên", "phòng", "thí", "nghiệm", "Xét", "Phân", "tích", "Nghiên", "cứu", "Thí"]},
    {"name": "Medical Technologist: Surgical Technology", "keywords": ["Kỹ", "thuật", "viên", "phẫu", "Phẫu", "Y", "tế", "Phòng", "mổ", "Hỗ", "trợ"]},
    {"name": "Dental Assistant and Therapist", "keywords": ["Trợ", "lý", "nha", "khoa", "Điều", "trị", "Răng", "Y", "tế", "Phụ", "tá"]},
    {"name": "Sports Professional", "keywords": ["Vận", "động", "viên", "chuyên", "nghiệp", "Thể", "thao", "Thi", "đấu", "Chuyên", "chất"]},
    {"name": "Physical Education Teacher", "keywords": ["Giáo", "viên", "thể", "dục", "Thể", "thao", "chất", "Dạy", "học"]},
    {"name": "Telecommunications Technician", "keywords": ["Kỹ", "thuật", "viên", "viễn", "thông", "Viễn", "Mạng", "Truyền", "Lắp", "đặt"]},
    {"name": "Fire-Fighter", "keywords": ["Lính", "cứu", "hỏa", "Cứu", "Khẩn", "cấp", "hộ", "An", "toàn"]},
    {"name": "Crop Grower", "keywords": ["Người", "trồng", "trọt", "Cây", "Nông", "nghiệp", "Canh", "tác", "Sản", "xuất"]},
    {"name": "Animal Husbandry Specialist", "keywords": ["Chuyên", "gia", "chăn", "nuôi", "Vật", "Gia", "súc", "Nông", "nghiệp", "Chăn"]},
    {"name": "Apiarist and Sericulturist", "keywords": ["Nuôi", "ong", "và", "tằm", "Ong", "Tằm", "Nông", "nghiệp", "Sản", "xuất"]},
    {"name": "Plumber and Pipe Fitter", "keywords": ["Thợ", "ống", "nước", "Hệ", "thống", "Lắp", "đặt", "Sửa", "chữa", "Xây", "dựng"]},
    {"name": "Tool and Die Maker", "keywords": ["Thợ", "làm", "khuôn", "Công", "cụ", "Chế", "tạo", "Cơ", "khí", "Khuôn", "mẫu"]},
    {"name": "Carpenter and Joiner", "keywords": ["Thợ", "mộc", "Đồ", "gỗ", "Xây", "dựng", "Mộc", "Nội", "thất"]},
    {"name": "HVAC Technician", "keywords": ["Kỹ", "thuật", "viên", "điều", "hòa", "Sưởi", "Làm", "mát", "Thông", "gió", "Lắp", "đặt"]},
    {"name": "Welder and Flame Cutter", "keywords": ["Thợ", "hàn", "Hàn", "Cắt", "kim", "loại", "Chế", "tạo", "Xây", "dựng"]},
    {"name": "Mechanical Engineering Technician (Ship Building)", "keywords": ["Kỹ", "thuật", "viên", "đóng", "tàu", "Đóng", "Tàu", "biển", "Hàng", "hải", "Cơ", "khí"]},
    {"name": "Mechanical Technician (Metal Working)", "keywords": ["Kỹ", "thuật", "viên", "kim", "loại", "Gia", "công", "Kim", "Cơ", "khí", "Sản", "xuất"]},
    {"name": "Auto Mechanic", "keywords": ["Thợ", "sửa", "ô", "tô", "Cơ", "khí", "Ô", "Sửa", "chữa", "Bảo", "dưỡng"]},
    {"name": "Mechanical Engineering Technician: Agricultural", "keywords": ["Kỹ", "thuật", "viên", "máy", "nông", "nghiệp", "Máy", "Cơ", "khí", "Canh", "tác", "Thiết", "bị"]},
    {"name": "Underwater Diver", "keywords": ["Thợ", "lặn", "Lặn", "Dưới", "nước", "Thám", "hiểm", "Hàng", "hải"]},
    {"name": "Locomotive Driver", "keywords": ["Lái", "tàu", "hỏa", "Đường", "sắt", "Tàu", "Vận", "tải", "Điều", "khiển"]},
    {"name": "Car, Taxi and Light Truck Driver", "keywords": ["Tài", "xế", "ô", "tô", "taxi", "Lái", "xe", "Vận", "tải", "Dịch", "vụ", "Giao", "thông"]},
    {"name": "Bus and Tram Driver", "keywords": ["Tài", "xế", "xe", "buýt", "Xe", "tram", "Giao", "thông", "công", "cộng", "Vận", "tải", "Hành", "khách"]},
    {"name": "Army officer", "keywords": ["Sĩ", "quan", "quân", "đội", "Quân", "sự", "Lãnh", "đạo", "Quốc", "phòng"]},
    {"name": "Forestry Worker", "keywords": ["Công", "nhân", "lâm", "nghiệp", "Rừng", "Khai", "thác", "Bảo", "vệ", "Làm", "việc", "ngoài", "trời"]},
    {"name": "Aquaculture and Fishery Worker", "keywords": ["Người", "nuôi", "trồng", "thủy", "sản", "Đánh", "bắt", "Thủy", "Cá", "Nuôi"]},
    {"name": "Mason", "keywords": ["Thợ", "nề", "Xây", "dựng", "Gạch", "đá", "Công", "trình"]},
    {"name": "Building Finisher", "keywords": ["Thợ", "hoàn", "thiện", "công", "trình", "Hoàn", "Xây", "dựng", "Nội", "thất", "Trang", "trí"]},
    {"name": "Painter", "keywords": ["Thợ", "sơn", "Sơn", "Xây", "dựng", "Trang", "trí", "Hoàn", "thiện"]},
    {"name": "Electrical Installation and Repair Worker", "keywords": ["Thợ", "điện", "Lắp", "đặt", "Sửa", "chữa", "Hệ", "thống", "Bảo", "trì"]},
    {"name": "Electronics and Telecommunications Installation and Repair Worker", "keywords": ["Thợ", "điện", "tử", "viễn", "thông", "Lắp", "đặt", "Sửa", "chữa", "Điện", "Viễn"]},
    {"name": "Assembler", "keywords": ["Thợ", "lắp", "ráp", "Lắp", "Sản", "xuất", "Nhà", "máy", "Linh", "kiện"]}
]

# Hàm chuẩn hóa tiêu đề sử dụng từ khóa
def normalize_title(title):
    if title is None:
        return "Unknown"
    
    # Chuẩn hóa title
    normalized = re.sub(r'\([^)]*\)', '', title)  # Xóa nội dung trong ngoặc ()
    normalized = re.sub(r'-.*$', '', normalized)   # Xóa nội dung sau dấu gạch ngang
    normalized = normalized.strip().lower()
    
    # Nếu title trống, trả về Unknown
    if not normalized:
        return "Unknown"
    
    best_match = None
    best_match_score = 0
    
    # Tìm kiếm theo số lượng từ khóa khớp
    for profession in professions:
        # Đếm số từ khóa trong title
        keywords_text = " ".join(profession["keywords"]).lower()
        
        # Tính điểm tương đồng
        ratio = fuzz.partial_ratio(normalized, keywords_text)
        token_sort_ratio = fuzz.token_sort_ratio(normalized, keywords_text)
        
        # Kết hợp các điểm số
        score = (ratio + token_sort_ratio) / 2
        
        if score > best_match_score:
            best_match_score = score
            best_match = profession["name"]
    
    if best_match_score >= 50:
        return best_match
    else:
        return "Unknown"

print(normalize_title("Trưởng Nhóm Kế Toán Quỹ"))


trưởng nhóm kế toán quỹ
Accountant


In [10]:
# New version - each keyword is a single word instead of a phrase
professions_single_words = []

for profession in professions:
    name = profession["name"]
    # Extract individual words from each keyword
    single_words = []
    for keyword in profession["keywords"]:
        # Split each keyword into individual words
        words = keyword.split()
        # Add each word to the list if not already present
        for word in words:
            if word not in single_words:
                single_words.append(word)
    
    # Create new profession entry with single words as keywords
    professions_single_words.append({
        "name": name,
        "keywords": single_words
    })
print(professions_single_words)

# Now professions_single_words contains the data with each keyword as a single word

[{'name': 'University Teacher', 'keywords': ['Giáo', 'sư', 'Đại', 'học', 'Giảng', 'dạy', 'Học', 'thuật', 'dục', 'Chuyên', 'viên']}, {'name': 'College Teacher', 'keywords': ['Giảng', 'viên', 'Cao', 'đẳng', 'Giáo', 'dục', 'Đào', 'tạo', 'dạy']}, {'name': 'Secondary Vocational and Professional School Teacher', 'keywords': ['Dạy', 'nghề', 'Chuyên', 'nghiệp', 'Giáo', 'viên', 'Đào', 'tạo', 'Kỹ', 'năng']}, {'name': 'Upper Secondary Education Teacher', 'keywords': ['Trung', 'học', 'phổ', 'thông', 'Giáo', 'viên', 'dục', 'Chương', 'trình']}, {'name': 'Lower Secondary Education Teacher', 'keywords': ['Trung', 'học', 'cơ', 'sở', 'Giáo', 'viên', 'dục', 'Thanh', 'thiếu', 'niên', 'Dạy']}, {'name': 'Foreign Language Teacher', 'keywords': ['Ngoại', 'ngữ', 'Giáo', 'viên', 'Ngôn', 'Quốc', 'tế', 'Giao', 'tiếp']}, {'name': 'Public Relations Officer', 'keywords': ['Quan', 'hệ', 'công', 'chúng', 'Truyền', 'thông', 'PR', 'Doanh', 'nghiệp', 'Hình', 'ảnh']}, {'name': 'Lawyer', 'keywords': ['Luật', 'sư', 'Pháp', 